In [0]:
dbutils.library.restartPython()

In [0]:

%sh

pip install azure-identity
pip install azure-monitor-query
pip install azure-mgmt-resourcegraph
pip install azure-mgmt-subscription
pip install netaddr


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: pip install --upgrade pip



### doc reference 

https://learn.microsoft.com/en-us/azure/network-watcher/traffic-analytics-schema?tabs=vnet#traffic-analytics-schema

In [0]:
vnet_logs = '''

let traffic_analytics_pip_table = NTAIpDetails
        | project ['PublicIP']=Ip, ['PIP_PublicIpDetails']=PublicIpDetails, ['PIP_ThreatType']=ThreatType, ['PIP_Location']=Location, ['PIP_Url']=Url;

NTANetAnalytics

| where FlowType != '' and SubType == 'FlowLog'

// filter grafana $__timeFrom, $__timeTo
//| where TimeGenerated between(startofday(datetime(2025-4-27)) .. endofday(datetime(2025-5-1)))
//| where TimeGenerated >= ago(1d)

| take 5000

| extend SrcPIP = substring(SrcPublicIps, 0, indexof(SrcPublicIps, "|"))
| extend DestPIP = substring(DestPublicIps, 0, indexof(DestPublicIps, "|"))

| extend SrcSubnetSplitted =  split(SrcSubnet, '/')
| extend SrcRG = tostring(SrcSubnetSplitted[0])
| extend SrcVNet = tostring(SrcSubnetSplitted[1])
| extend SrcSubnetName = tostring(SrcSubnetSplitted[2])

| extend DestSubnetSplitted =  split(DestSubnet, '/')
| extend DestRG = tostring(DestSubnetSplitted[0])
| extend DestVNet = tostring(DestSubnetSplitted[1])
| extend DestSubnetName = tostring(DestSubnetSplitted[2])



// resolve Azure Public IP from NTAIpDetails table
| join kind=leftouter
(
    traffic_analytics_pip_table
| project ['SrcPIP_Ip']=PublicIP, ['SrcPIP_PublicIpDetails']=PIP_PublicIpDetails, ['SrcPIP_ThreatType']=PIP_ThreatType, ['SrcPIP_Location']=PIP_Location, ['SrcPIP_Url']=PIP_Url
)
on $left.SrcPIP == $right.SrcPIP_Ip

| join kind=leftouter
    (
        traffic_analytics_pip_table
        | project ['DestPIP_Ip']=PublicIP, ['DestPIP_PublicIpDetails']=PIP_PublicIpDetails, ['DestPIP_ThreatType']=PIP_ThreatType, ['DestPIP_Location']=PIP_Location, ['DestPIP_Url']=PIP_Url
    )
on $left.DestPIP == $right.DestPIP_Ip

// set SrcPIP_Ip if exist
| extend SrcIp = iif(SrcPIP_Ip != '', SrcPIP_Ip, SrcIp)
// set DestPIP_Ip if exist
| extend DestIp = strcat(iif(DestPIP_Ip != '', DestPIP_Ip, DestIp), ':', DestPort)

//check "location" of src node
// | extend SrcNodeInOrOutVNet =   iif(SrcIp != '' and FlowType != 'MaliciousFlow', 'NodeInsideVNet',
//                                   iif(FlowType == 'AzurePublic' and SrcPublicIps != '', 'NodeOutsideVNet_PaaS',
//                                     iif(FlowType == 'ExternalPublic' and SrcPublicIps != '', 'NodeOutsideAzure',
//                                       //adversary from outside of Azure
//                                       iif (FlowType =='MaliciousFlow' and FlowDirection == 'Inbound' and SrcIp != '', 'NodeMalicious',
//                                         //adversary from inside of Azure
//                                         iif(FlowType =='MaliciousFlow' and FlowDirection == 'Outbound', 'NodeInsideVNet', 'NodeUnknown')
//                                 )
//                               )
//                             )
//                           )

//check "location" of dest node
// | extend DestNodeInOrOutVNet = iif(DestIp != '' and FlowType != 'MaliciousFlow', 'NodeInsideVNet',
//                             iif(FlowType in ('AzurePublic') and DestPublicIps != '', 'NodeOutsideVNet_PaaS',
//                                 iif(FlowType == 'ExternalPublic' and DestPublicIps != '', 'NodeOutsideAzure',
//                                     //adversary from outside of Azure
//                                     iif(FlowType == 'MaliciousFlow' and FlowDirection== 'Inbound' and DestIp != '', 'NodeInsideVNet',
//                                         //adversary from inside of Azure
//                                         iif(FlowType == 'MaliciousFlow' and FlowDirection== 'Outbound', 'NodeMalicious', 'NodeUnknown')
//                                 )
//                               )
//                             )
//                           )


// resolve SrcName as private endpoint name if traffic is private endpoint
// *note: private endpoint is Source and not destination
| extend PrivateEndpointName = split(PrivateEndpointResourceId, '/')[-1]
| extend PrivateLinkName = split(PrivateEndpointResourceId, '/')[-2]

| extend SrcName = iif(SrcApplicationGateway != '', SrcApplicationGateway,
                     iif(SrcLoadBalancer != '', SrcLoadBalancer,
                        iif(SrcPIP_PublicIpDetails != '', SrcPIP_PublicIpDetails,
                          iif(SrcSubnetName == 'azurebastionsubnet', 'bastion-vm',
                           iif(PrivateEndpointResourceId != '' and PrivateLinkResourceId != '',  PrivateEndpointName,
                             iif(SrcNic startswith 'unknown', strcat('managed vm in ', iif(SrcSubnetName has 'subnet', SrcSubnetName, strcat(SrcSubnetName, ' subnet'))),
                               iif(SrcVm != '', SrcVm, '-' )))))))

| extend SrcName = iif(indexof(SrcName, '/',0) > 0, split(SrcName, '/')[-1], SrcName)


| extend DestName = iif(DestApplicationGateway != '', DestApplicationGateway,
                        iif(DestLoadBalancer != '', DestLoadBalancer,
                            iif(DestPIP_PublicIpDetails != '', DestPIP_PublicIpDetails,
                               iif(DestSubnetName == 'azurebastionsubnet', 'bastion-vm',
                                iif(PrivateEndpointResourceId != '' and PrivateLinkResourceId != '',  PrivateLinkName,
                                  iif(DestNic startswith 'unknown', strcat('managed vm in ', iif(DestSubnetName has 'subnet', DestSubnetName, strcat(DestSubnetName, ' subnet'))),
                                    iif(DestVm != '',  DestVm, '' )))))))


| extend DestName = iif(indexof(DestName, '/',0) > 0, split(DestName, '/')[-1], DestName)


| extend protocol = tolower(strcat(L4Protocol,  iif(L7Protocol != 'Unknown', strcat(':', L7Protocol), '')))


| summarize BytesSrcToDest = max(BytesSrcToDest), BytesDestToSrc = max(BytesDestToSrc)
            by  FlowType, FlowDirection, ConnectionType, protocol,
                    SrcSubscription, SrcRG, SrcVNet, SrcSubnetName , SrcIp, SrcName,
                    DestSubscription, DestRG, DestVNet, DestSubnetName, DestName,
                    SrcPIP_ThreatType, SrcPIP_Location, SrcPIP_Url,
                    DestIp, DestPIP_ThreatType, DestPIP_Location, DestPIP_Url


| extend SrcToDestDataSize = format_bytes(BytesSrcToDest, 2)
| extend DestToSrcDataSize = format_bytes(BytesDestToSrc, 2)

| distinct FlowType, FlowDirection, ConnectionType, Protocol=protocol,
        SrcSubscription, SrcRG, SrcVNet, SrcSubnetName , SrcIp, SrcName, SrcToDestDataSize,
        DestSubscription, DestRG, DestVNet, DestSubnetName, DestName, DestToSrcDataSize,
        SrcPIP_ThreatType, SrcPIP_Location, SrcPIP_Url,
        DestIp, DestPIP_ThreatType, DestPIP_Location, DestPIP_Url

'''


### main df - vnet flow logs, drop duplicates and fill null

In [0]:
from azure.monitor.query import LogsQueryClient
from azure.monitor.query._models import LogsQueryPartialResult, LogsTable, LogsQueryStatus
from azure.identity import DefaultAzureCredential
import datetime
from datetime import datetime, timedelta, timezone
import pandas as pd
import math
import json
from azure.core.exceptions import HttpResponseError

credential = DefaultAzureCredential()
client = LogsQueryClient(credential)

start_time=datetime(2025, 4, 27, tzinfo=timezone.utc)
end_time=datetime(2025, 5, 1, tzinfo=timezone.utc)

df = pd.DataFrame()

try:
    response = client.query_workspace(
                workspace_id= 'c596239e-beda-4999-a79f-58e34d9881e4',
                query=vnet_logs,
                #timespan=None #timedelta(minutes=30)
                timespan=(start_time, end_time)
            )

    if response.status == LogsQueryStatus.SUCCESS:
        table = response.tables[0]
        df = pd.DataFrame(data=table.rows, columns=table.columns)

except HttpResponseError as err:
    print("something fatal happened")
    print(err)



In [0]:

# drop all logs where they are: Inbound-ExternalPublic and found in NTAIpDetails table
externalpublic_inbound_NTAIpDetails_match_indexes = df[(df['FlowDirection']=='Inbound') & (df['FlowType']=='ExternalPublic') & (df['SrcName'] != '')].index

tempdf = df.drop(externalpublic_inbound_NTAIpDetails_match_indexes)

# update SrcName by Subnet
                                               
tempdf['SrcName'] = tempdf.apply(lambda x: x['SrcSubnetName'] if x['SrcName']=='' else x['SrcName'], axis=1)

maindf = tempdf.drop_duplicates(subset=['SrcName', 'DestName'])

maindf.fillna('', inplace=True)

print(len(maindf.index))

print(maindf['FlowType'].unique())

maindf



94
['AzurePublic' 'ExternalPublic' 'UnknownPrivate' 'InterVNet' 'IntraVNet'
 'MaliciousFlow']


/root/.ipykernel/1696/command-5477959247676144-1664285272:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maindf.fillna('', inplace=True)


,FlowType,FlowDirection,ConnectionType,Protocol,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
0,AzurePublic,Outbound,,tcp,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.4,vnetgw-cwhd-onpremise-VM-192.166.0.4,928 Bytes,,,,,AzureCloud,0 Bytes,,,,13.65.254.147:22215,,southcentralus,
1,AzurePublic,Outbound,,tcp:https,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-ingress,apim,172.16.2.5,managed vm in apim subnet,32.42 MB,,,,,AzureKeyVault,33.69 MB,,,,13.67.8.104:443,,southeastasia,
2,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.5,vnetgw-cwhd-onpremise-VM-192.166.0.5,592.33 KB,,,,,AzureMonitor,253.69 KB,,,,13.71.196.56:1886,,westcentralus,
3,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,172.18.0.5,vpngw-cwhd-s2s-azure-VM-172.18.0.5,570.76 KB,,,,,AzureMonitor,193.54 KB,,,,13.71.196.56:1886,,westcentralus,
4,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.4,vnetgw-cwhd-onpremise-VM-192.166.0.4,493.9 KB,,,,,AzureMonitor,253.59 KB,,,,13.71.196.56:1886,,westcentralus,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2353,AzurePublic,Inbound,,tcp:pcsync-https,,,,,20.43.128.73,GatewayManager,840 Bytes,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,vnetgw-cwhd-onpremise-VM-192.166.0.4,264 Bytes,,southeastasia,,192.166.0.4:8443,,,
2354,AzurePublic,Inbound,,tcp:pcsync-http,,,,,20.43.128.73,GatewayManager,26.96 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,vnetgw-cwhd-onpremise-VM-192.166.0.5,26.88 KB,,southeastasia,,192.166.0.5:8444,,,
2359,AzurePublic,Inbound,,tcp:pcsync-https,,,,,20.195.86.98,GatewayManager,27.34 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,vpngw-cwhd-s2s-azure-VM-172.18.0.4,26.61 KB,,southeastasia,,172.18.0.4:8443,,,
3705,AzurePublic,Inbound,,tcp:pcsync-http,,,,,20.195.86.100,GatewayManager,28.48 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,vpngw-cwhd-s2s-azure-VM-172.18.0.5,27.29 KB,,southeastasia,,172.18.0.5:8444,,,



### resolve vnet and subnet name of UnknownPrivate flow type 

In [0]:

from azure.identity import DefaultAzureCredential
from azure.mgmt.subscription import SubscriptionClient
from azure.mgmt.resourcegraph import ResourceGraphClient
from azure.mgmt.resourcegraph.models import *
import ipaddress

query = '''
resources
| where type == "microsoft.network/virtualnetworks"
| extend VNet = name
| mv-expand addressPrefixes = properties.addressSpace.addressPrefixes
| mv-expand subnet = properties.subnets
| project VNet, VNetAddressSpace = addressPrefixes, SubnetName=subnet.name, SubnetAddressPrefix=subnet.properties.addressPrefix
'''

azcred = DefaultAzureCredential()

# subscription ids
sub_client = SubscriptionClient(azcred)
subscription_list = sub_client.subscriptions.list()
subids = [item.subscription_id for item in subscription_list]

resourcegraph_client = ResourceGraphClient(
        credential=DefaultAzureCredential()
    )

    # Create Azure Resource Graph client and set options
query = QueryRequest(
            query=query,
            subscriptions=subids,
            options=QueryRequestOptions(
                result_format=ResultFormat.TABLE
            )
        )
query_response = resourcegraph_client.resources(query)


data_cols = [x['name'] for x in query_response.data['columns']]
data_row = query_response.data['rows']
vnet_subnet_names = pd.DataFrame(data_row, columns=data_cols)


# resolve vnet name and subnet name in maind from existing vnets
unknownprivate_df = maindf[maindf['FlowType'] == 'UnknownPrivate']

for index, ukprow in unknownprivate_df.iterrows():
    srcip = ukprow['SrcIp']
    destip = ukprow['DestIp']

    for index, vnet_subnet in vnet_subnet_names.iterrows():
        subnet_cid = vnet_subnet['SubnetAddressPrefix']
        
        try:

            if ipaddress.ip_address(srcip) in ipaddress.ip_network(subnet_cid):
                unknownprivate_df['SrcVNet'] = vnet_subnet['VNet']
                unknownprivate_df['SrcSubnetName'] = vnet_subnet['SubnetName']
                unknownprivate_df['SrcName'] = 'vm in ' +  unknownprivate_df["SrcSubnetName"]

            if ipaddress.ip_address(destip) in ipaddress.ip_network(subnet_cid):
                unknownprivate_df['DestName'] = vnet_subnet['VNet']
                unknownprivate_df['DestSubnetName'] = vnet_subnet['SubnetName']
                unknownprivate_df['DestName'] = 'vm in ' +  unknownprivate_df["SrcSubnetName"]

        except Exception as e:
            continue


maindf.update(unknownprivate_df)
maindf


/root/.ipykernel/1696/command-6510015665464912-765320633:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknownprivate_df['SrcVNet'] = vnet_subnet['VNet']
/root/.ipykernel/1696/command-6510015665464912-765320633:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknownprivate_df['SrcSubnetName'] = vnet_subnet['SubnetName']
/root/.ipykernel/1696/command-6510015665464912-765320633:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

,FlowType,FlowDirection,ConnectionType,Protocol,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
0,AzurePublic,Outbound,,tcp,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.4,vnetgw-cwhd-onpremise-VM-192.166.0.4,928 Bytes,,,,,AzureCloud,0 Bytes,,,,13.65.254.147:22215,,southcentralus,
1,AzurePublic,Outbound,,tcp:https,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-ingress,apim,172.16.2.5,managed vm in apim subnet,32.42 MB,,,,,AzureKeyVault,33.69 MB,,,,13.67.8.104:443,,southeastasia,
2,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.5,vnetgw-cwhd-onpremise-VM-192.166.0.5,592.33 KB,,,,,AzureMonitor,253.69 KB,,,,13.71.196.56:1886,,westcentralus,
3,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,172.18.0.5,vpngw-cwhd-s2s-azure-VM-172.18.0.5,570.76 KB,,,,,AzureMonitor,193.54 KB,,,,13.71.196.56:1886,,westcentralus,
4,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.4,vnetgw-cwhd-onpremise-VM-192.166.0.4,493.9 KB,,,,,AzureMonitor,253.59 KB,,,,13.71.196.56:1886,,westcentralus,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2353,AzurePublic,Inbound,,tcp:pcsync-https,,,,,20.43.128.73,GatewayManager,840 Bytes,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,vnetgw-cwhd-onpremise-VM-192.166.0.4,264 Bytes,,southeastasia,,192.166.0.4:8443,,,
2354,AzurePublic,Inbound,,tcp:pcsync-http,,,,,20.43.128.73,GatewayManager,26.96 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,vnetgw-cwhd-onpremise-VM-192.166.0.5,26.88 KB,,southeastasia,,192.166.0.5:8444,,,
2359,AzurePublic,Inbound,,tcp:pcsync-https,,,,,20.195.86.98,GatewayManager,27.34 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,vpngw-cwhd-s2s-azure-VM-172.18.0.4,26.61 KB,,southeastasia,,172.18.0.4:8443,,,
3705,AzurePublic,Inbound,,tcp:pcsync-http,,,,,20.195.86.100,GatewayManager,28.48 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,vpngw-cwhd-s2s-azure-VM-172.18.0.5,27.29 KB,,southeastasia,,172.18.0.5:8444,,,


In [0]:
maindf[maindf['FlowType'] == 'UnknownPrivate']

,FlowType,FlowDirection,ConnectionType,Protocol,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
403,UnknownPrivate,Inbound,,tcp:discard,,,vnet-cwhd-s2s-azure,AzureBastionSubnet,172.18.0.69,vm in AzureBastionSubnet,14.53 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-management,vms,vm-build-agent-win-server-1,0 Bytes,,,,10.0.1.75:9,,,



### create echarts nodes array

In [0]:
    # {
    #   name: 'CNF',
    #   id: 'CNF',
    #   category: 1,
    #   ip: '12.20.11.23',
    #   subnet: "some text",
    #   vnet: '123'
    # },



# create nodes csv for IntraVNet

src_nodes_final_df = pd.DataFrame()
src_nodes_final_df['id'] = maindf['SrcIp']
src_nodes_final_df['name'] = maindf['SrcName']
src_nodes_final_df['category'] = maindf['FlowType']
src_nodes_final_df['ip'] = maindf['SrcIp']
src_nodes_final_df['subnet'] = maindf['SrcSubnetName']
src_nodes_final_df['vnet'] = maindf['SrcVNet']
src_nodes_final_df['srcPIPLocation'] = maindf['SrcPIP_Location']


dest_nodes_final_df = pd.DataFrame()
dest_nodes_final_df['id'] = maindf['DestIp']
dest_nodes_final_df['name'] = maindf['DestName']
dest_nodes_final_df['category'] = maindf['FlowType']
dest_nodes_final_df['ip'] = maindf['DestIp']
dest_nodes_final_df['subnet'] = maindf['DestSubnetName']
dest_nodes_final_df['vnet'] = maindf['DestVNet']
src_nodes_final_df['destPIPLocation'] = maindf['DestPIP_Location']

# # #id,title,subtitle,mainstat,secondarystat,color
combined_list = [src_nodes_final_df, dest_nodes_final_df]

final_nodes_df = pd.concat(combined_list)

final_nodes_df = final_nodes_df.drop_duplicates(subset=['id'])

final_nodes_json_str =  final_nodes_df.to_json(orient='records')

final_nodes_json_obj = json.loads(final_nodes_json_str)


print(final_nodes_json_str)


[{"id":"192.166.0.4","name":"vnetgw-cwhd-onpremise-VM-192.166.0.4","category":"AzurePublic","ip":"192.166.0.4","subnet":"gatewaysubnet","vnet":"vnet-cwh-s2s-onpremise","srcPIPLocation":"","destPIPLocation":"southcentralus"},{"id":"172.16.2.5","name":"managed vm in apim subnet","category":"AzurePublic","ip":"172.16.2.5","subnet":"apim","vnet":"vnet-ingress","srcPIPLocation":"","destPIPLocation":"southeastasia"},{"id":"192.166.0.5","name":"vnetgw-cwhd-onpremise-VM-192.166.0.5","category":"AzurePublic","ip":"192.166.0.5","subnet":"gatewaysubnet","vnet":"vnet-cwh-s2s-onpremise","srcPIPLocation":"","destPIPLocation":"westcentralus"},{"id":"172.18.0.5","name":"vpngw-cwhd-s2s-azure-VM-172.18.0.5","category":"AzurePublic","ip":"172.18.0.5","subnet":"gatewaysubnet","vnet":"vnet-cwhd-s2s-azure","srcPIPLocation":"","destPIPLocation":"westcentralus"},{"id":"172.18.0.4","name":"vpngw-cwhd-s2s-azure-VM-172.18.0.4","category":"AzurePublic","ip":"172.18.0.4","subnet":"gatewaysubnet","vnet":"vnet-cwhd-


### create echarts edges array

In [0]:

    # {
    #   source: 'CNF',
    #   target: 'SCPC',
    #   src_to_dest_data_size: '10 KB',
    #   dest_to_srct_data_size: '22 KB',
    #   flowType: 'Azure Public PaaS',
    #   label: {
    #     show: true,
    #     formatter: (params) => params.data.src_to_dest_data_size + '-> <- ' + params.data.dest_to_srct_data_size + '\n' + params.data.flowType,
    #     fontSize: 10
    #   },
    #   lineStyle: {
    #     curveness: 0.1
    #   }
    # }

# create edge

edges_df = pd.DataFrame()
edges_df['source'] = tempdf['SrcIp']
edges_df['target'] = tempdf['DestIp']
# edges_df['value'] = tempdf['SrcToDestDataSize'] + '-> <-' + tempdf['DestToSrcDataSize'] + '<div> flowtype: ' + tempdf['FlowType'] + '</div>' + '<div> protocol: ' +  tempdf['protocol'] + '/div>'
edges_df['src_to_dest_data_size'] = tempdf['SrcToDestDataSize']
edges_df['dest_to_srct_data_size'] = tempdf['DestToSrcDataSize']
edges_df['flowType'] = tempdf['FlowType']
edges_df['protocol'] = tempdf['Protocol']
edges_df['connectionType'] = tempdf['ConnectionType']

edges_df = edges_df.drop_duplicates(subset=['source', 'target'])

final_edges_json_str =  edges_df.to_json(orient='records')

final_edges_json_obj = json.loads(final_edges_json_str)

print(final_edges_json_str)


[{"source":"192.166.0.4","target":"13.65.254.147:22215","src_to_dest_data_size":"928 Bytes","dest_to_srct_data_size":"0 Bytes","flowType":"AzurePublic","protocol":"tcp","connectionType":""},{"source":"172.16.2.5","target":"13.67.8.104:443","src_to_dest_data_size":"32.42 MB","dest_to_srct_data_size":"33.69 MB","flowType":"AzurePublic","protocol":"tcp:https","connectionType":""},{"source":"192.166.0.5","target":"13.71.196.56:1886","src_to_dest_data_size":"592.33 KB","dest_to_srct_data_size":"253.69 KB","flowType":"AzurePublic","protocol":"tcp:leoip","connectionType":""},{"source":"172.18.0.5","target":"13.71.196.56:1886","src_to_dest_data_size":"570.76 KB","dest_to_srct_data_size":"193.54 KB","flowType":"AzurePublic","protocol":"tcp:leoip","connectionType":""},{"source":"192.166.0.4","target":"13.71.196.56:1886","src_to_dest_data_size":"493.9 KB","dest_to_srct_data_size":"253.59 KB","flowType":"AzurePublic","protocol":"tcp:leoip","connectionType":""},{"source":"172.18.0.4","target":"13.7

### categories result

In [0]:

# category

# unique_src_node_location = maindf['SrcNodeInOrOutVNet'].unique()
# unique_dest_node_location = maindf['DestNodeInOrOutVNet'].unique()

# unique_categories = list(set(unique_src_node_location) | set(unique_dest_node_location))

# result = pd.DataFrame()
# result['name'] = unique_categories


result = pd.DataFrame()
result['name'] = maindf['FlowType'].unique()


# for x in unique_src_node_location + unique_dest_node_location:
#     result.add(x)

# result = list(result)
unique_categories =  result.to_json(orient='records')

print(unique_categories)

[{"name":"AzurePublic"},{"name":"ExternalPublic"},{"name":"UnknownPrivate"},{"name":"InterVNet"},{"name":"IntraVNet"},{"name":"MaliciousFlow"}]


### node and edges as single result

In [0]:
final_json = {
    "nodes": final_nodes_json_obj,
    "edges": final_edges_json_obj,
    "categories": unique_categories
}

json_str = json.dumps(final_json)

with open('nodes_edges.txt', "w") as f:
    f.write(json_str)


### resolve vnet and subnet names of UnknownPrivate flow type

In [0]:




unknownprivate_df = maindf[maindf['FlowType'] == 'UnknownPrivate']

for index, ukprow in unknownprivate_df.iterrows():
    srcip = ukprow['SrcIp']
    destip = ukprow['DestIp']

    for index, vnet_subnet in vnet_subnet_names.iterrows():
        subnet_cid = vnet_subnet['SubnetAddressPrefix']
        
        try:

            if ipaddress.ip_address(srcip) in ipaddress.ip_network(subnet_cid):
                unknownprivate_df['SrcVNet'] = vnet_subnet['VNet']
                unknownprivate_df['SrcSubnetName'] = vnet_subnet['SubnetName']
                unknownprivate_df['SrcName'] = 'vm in ' +  unknownprivate_df["SrcSubnetName"]

            if ipaddress.ip_address(destip) in ipaddress.ip_network(subnet_cid):
                unknownprivate_df['DestName'] = vnet_subnet['VNet']
                unknownprivate_df['DestSubnetName'] = vnet_subnet['SubnetName']
                unknownprivate_df['DestName'] = 'vm in ' +  unknownprivate_df["SrcSubnetName"]

        except Exception as e:
            continue


maindf.update(unknownprivate_df)
maindf


/root/.ipykernel/1696/command-4954130380456555-3297756707:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maindf.update(unknownprivate_df)
/root/.ipykernel/1696/command-4954130380456555-3297756707:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maindf.update(unknownprivate_df)
/root/.ipykernel/1696/command-4954130380456555-3297756707:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,Src_NTAIpDetails,Dest_NTAIpDetails,FlowType,FlowDirection,ConnectionType,Protocol,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
0,,,UnknownPrivate,Inbound,,tcp:http,,,vnet-cwhd-s2s-azure,AzureBastionSubnet,172.18.0.69,vm in AzureBastionSubnet,102.19 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-management,vms,vm-build-agent-win-server-1,138.75 KB,,,,10.0.1.75:80,,,


In [0]:
maindf

,Src_NTAIpDetails,Dest_NTAIpDetails,FlowType,FlowDirection,ConnectionType,Protocol,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
0,,,UnknownPrivate,Inbound,,tcp:http,,,,,172.18.0.69,-,102.19 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-management,vms,vm-build-agent-win-server-1,138.75 KB,,,,10.0.1.75:80,,,


/root/.ipykernel/1696/command-6661004786289939-1518921000:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maindf.update(unknownprivate_df)
/root/.ipykernel/1696/command-6661004786289939-1518921000:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maindf.update(unknownprivate_df)
/root/.ipykernel/1696/command-6661004786289939-1518921000:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,Src_NTAIpDetails,Dest_NTAIpDetails,FlowType,FlowDirection,ConnectionType,Protocol,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
0,,,UnknownPrivate,Inbound,,tcp:http,,,vnet-cwhd-s2s-azure,AzureBastionSubnet,172.18.0.69,vm in AzureBastionSubnet,102.19 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-management,vms,vm-build-agent-win-server-1,138.75 KB,,,,10.0.1.75:80,,,



### unique source info for grafana filter

In [0]:

source_filter = {"nodes": [{"id": "192.166.0.4", "name": "vnetgw-cwhd-onpremise-VM-192.166.0.4", "category": "AzurePublic", "ip": "192.166.0.4", "subnet": "gatewaysubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": "", "destPIPLocation": "southcentralus"}, {"id": "172.16.2.5", "name": "managed vm in apim subnet", "category": "AzurePublic", "ip": "172.16.2.5", "subnet": "apim", "vnet": "vnet-ingress", "srcPIPLocation": "", "destPIPLocation": "southeastasia"}, {"id": "192.166.0.5", "name": "vnetgw-cwhd-onpremise-VM-192.166.0.5", "category": "AzurePublic", "ip": "192.166.0.5", "subnet": "gatewaysubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": "", "destPIPLocation": "westcentralus"}, {"id": "172.18.0.5", "name": "vpngw-cwhd-s2s-azure-VM-172.18.0.5", "category": "AzurePublic", "ip": "172.18.0.5", "subnet": "gatewaysubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": "", "destPIPLocation": "westcentralus"}, {"id": "172.18.0.4", "name": "vpngw-cwhd-s2s-azure-VM-172.18.0.4", "category": "AzurePublic", "ip": "172.18.0.4", "subnet": "gatewaysubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": "", "destPIPLocation": "westcentralus"}, {"id": "10.0.1.5", "name": "bastion-vm", "category": "AzurePublic", "ip": "10.0.1.5", "subnet": "azurebastionsubnet", "vnet": "vnet-management", "srcPIPLocation": "", "destPIPLocation": "southeastasia"}, {"id": "172.18.0.69", "name": "bastion-vm", "category": "AzurePublic", "ip": "172.18.0.69", "subnet": "azurebastionsubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": "", "destPIPLocation": "southeastasia"}, {"id": "192.166.0.68", "name": "bastion-vm", "category": "AzurePublic", "ip": "192.166.0.68", "subnet": "azurebastionsubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": "", "destPIPLocation": "southeastasia"}, {"id": "172.18.1.4", "name": "vm-cwhd-s2s-azure", "category": "AzurePublic", "ip": "172.18.1.4", "subnet": "vm", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": "", "destPIPLocation": "eastus"}, {"id": "10.0.1.75", "name": "vm-build-agent-win-server-1", "category": "AzurePublic", "ip": "10.0.1.75", "subnet": "vms", "vnet": "vnet-management", "srcPIPLocation": "", "destPIPLocation": "southeastasia"}, {"id": "10.0.1.4", "name": "bastion-vm", "category": "AzurePublic", "ip": "10.0.1.4", "subnet": "azurebastionsubnet", "vnet": "vnet-management", "srcPIPLocation": "", "destPIPLocation": ""}, {"id": "172.18.0.68", "name": "bastion-vm", "category": "ExternalPublic", "ip": "172.18.0.68", "subnet": "azurebastionsubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": "", "destPIPLocation": "sg"}, {"id": "192.166.0.69", "name": "bastion-vm", "category": "IntraVNet", "ip": "192.166.0.69", "subnet": "azurebastionsubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": "", "destPIPLocation": ""}, {"id": "196.251.70.87", "name": "-", "category": "MaliciousFlow", "ip": "196.251.70.87", "subnet": "", "vnet": "", "srcPIPLocation": "", "destPIPLocation": ""}, {"id": "52.187.163.69", "name": "AzureCloud", "category": "AzurePublic", "ip": "52.187.163.69", "subnet": "", "vnet": "", "srcPIPLocation": "southeastasia", "destPIPLocation": ""}, {"id": "40.80.36.0", "name": "AzureCloud", "category": "AzurePublic", "ip": "40.80.36.0", "subnet": "", "vnet": "", "srcPIPLocation": "koreacentral", "destPIPLocation": ""}, {"id": "40.90.185.46", "name": "AzureApiManagement", "category": "AzurePublic", "ip": "40.90.185.46", "subnet": "", "vnet": "", "srcPIPLocation": "southeastasia", "destPIPLocation": ""}, {"id": "20.43.128.73", "name": "GatewayManager", "category": "AzurePublic", "ip": "20.43.128.73", "subnet": "", "vnet": "", "srcPIPLocation": "southeastasia", "destPIPLocation": ""}, {"id": "20.195.86.98", "name": "GatewayManager", "category": "AzurePublic", "ip": "20.195.86.98", "subnet": "", "vnet": "", "srcPIPLocation": "southeastasia", "destPIPLocation": ""}, {"id": "20.195.86.100", "name": "GatewayManager", "category": "AzurePublic", "ip": "20.195.86.100", "subnet": "", "vnet": "", "srcPIPLocation": "southeastasia", "destPIPLocation": ""}, {"id": "52.163.205.184", "name": "AzureCloud", "category": "AzurePublic", "ip": "52.163.205.184", "subnet": "", "vnet": "", "srcPIPLocation": "southeastasia", "destPIPLocation": ""}, {"id": "13.65.254.147:22215", "name": "AzureCloud", "category": "AzurePublic", "ip": "13.65.254.147:22215", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "13.67.8.104:443", "name": "AzureKeyVault", "category": "AzurePublic", "ip": "13.67.8.104:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "13.71.196.56:1886", "name": "AzureMonitor", "category": "AzurePublic", "ip": "13.71.196.56:1886", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "13.71.196.56:443", "name": "AzureMonitor", "category": "AzurePublic", "ip": "13.71.196.56:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.150.28.164:443", "name": "AzureStorage", "category": "AzurePublic", "ip": "20.150.28.164:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.189.173.1:443", "name": "OneDsCollector", "category": "AzurePublic", "ip": "20.189.173.1:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.209.110.8:443", "name": "AzureStorage", "category": "AzurePublic", "ip": "20.209.110.8:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.60.105.43:443", "name": "AzureStorage", "category": "AzurePublic", "ip": "20.60.105.43:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.60.136.97:443", "name": "AzureStorage", "category": "AzurePublic", "ip": "20.60.136.97:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "52.154.220.73:443", "name": "AzureCloud", "category": "AzurePublic", "ip": "52.154.220.73:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "13.69.109.238:12000", "name": "AzureMonitor", "category": "AzurePublic", "ip": "13.69.109.238:12000", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.150.28.228:443", "name": "AzureStorage", "category": "AzurePublic", "ip": "20.150.28.228:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.190.163.18:443", "name": "AzureIdentity", "category": "AzurePublic", "ip": "20.190.163.18:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "23.98.114.99:443", "name": "AzureMonitor", "category": "AzurePublic", "ip": "23.98.114.99:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "13.107.42.16:443", "name": "AzureFrontDoor", "category": "AzurePublic", "ip": "13.107.42.16:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.195.81.86:443", "name": "AzureEventHub", "category": "AzurePublic", "ip": "20.195.81.86:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.42.73.26:443", "name": "OneDsCollector", "category": "AzurePublic", "ip": "20.42.73.26:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.44.239.154:443", "name": "AzureCloud", "category": "AzurePublic", "ip": "20.44.239.154:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "23.15.147.118:80", "name": "akamai technologies  inc.", "category": "ExternalPublic", "ip": "23.15.147.118:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "23.210.96.161:80", "name": "akamai technologies  inc.", "category": "ExternalPublic", "ip": "23.210.96.161:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "52.148.114.188:123", "name": "AzureCloud", "category": "AzurePublic", "ip": "52.148.114.188:123", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.190.144.162:443", "name": "AzureIdentity", "category": "AzurePublic", "ip": "20.190.144.162:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.42.73.25:443", "name": "OneDsCollector", "category": "AzurePublic", "ip": "20.42.73.25:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "146.75.46.172:80", "name": "fastly  inc.", "category": "ExternalPublic", "ip": "146.75.46.172:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.195.65.39:1433", "name": "AzureSQL", "category": "AzurePublic", "ip": "20.195.65.39:1433", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "52.168.117.168:443", "name": "OneDsCollector", "category": "AzurePublic", "ip": "52.168.117.168:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "13.107.246.59:443", "name": "AzureFrontDoor.Frontend", "category": "AzurePublic", "ip": "13.107.246.59:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "204.79.197.203:80", "name": "AzureFrontDoor", "category": "AzurePublic", "ip": "204.79.197.203:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.242.39.171:443", "name": "AzureUpdateDelivery", "category": "AzurePublic", "ip": "20.242.39.171:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "52.168.117.175:443", "name": "OneDsCollector", "category": "AzurePublic", "ip": "52.168.117.175:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "13.107.253.59:443", "name": "AzureFrontDoor.Frontend", "category": "AzurePublic", "ip": "13.107.253.59:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "13.95.31.18:443", "name": "AzureUpdateDelivery", "category": "AzurePublic", "ip": "13.95.31.18:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "23.36.48.35:443", "name": "akamai technologies  inc.", "category": "ExternalPublic", "ip": "23.36.48.35:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "23.210.250.120:80", "name": "akamai international bv", "category": "ExternalPublic", "ip": "23.210.250.120:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "23.61.202.49:80", "name": "akamai technologies  inc.", "category": "ExternalPublic", "ip": "23.61.202.49:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "43.174.32.117:80", "name": "6 collyer quay", "category": "ExternalPublic", "ip": "43.174.32.117:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "184.87.193.149:80", "name": "akamai technologies  inc.", "category": "ExternalPublic", "ip": "184.87.193.149:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "43.174.14.43:80", "name": "6 collyer quay", "category": "ExternalPublic", "ip": "43.174.14.43:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "23.61.202.42:80", "name": "akamai technologies  inc.", "category": "ExternalPublic", "ip": "23.61.202.42:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "34.107.245.121:443", "name": "google", "category": "ExternalPublic", "ip": "34.107.245.121:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "101.33.26.246:80", "name": "6 collyer quay", "category": "ExternalPublic", "ip": "101.33.26.246:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.10.127.193:443", "name": "MicrosoftDefenderForEndpoint", "category": "AzurePublic", "ip": "20.10.127.193:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "23.98.113.115:443", "name": "AzureMonitor", "category": "AzurePublic", "ip": "23.98.113.115:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.195.81.84:5671", "name": "AzureEventHub", "category": "AzurePublic", "ip": "20.195.81.84:5671", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.10.127.192:443", "name": "MicrosoftDefenderForEndpoint", "category": "AzurePublic", "ip": "20.10.127.192:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "23.36.50.36:443", "name": "akamai technologies  inc.", "category": "ExternalPublic", "ip": "23.36.50.36:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "43.175.139.73:80", "name": "16 collyer quay   18-29 income at raffles", "category": "ExternalPublic", "ip": "43.175.139.73:80", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "52.123.252.3:443", "name": "microsoft corporation", "category": "ExternalPublic", "ip": "52.123.252.3:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "20.12.23.50:443", "name": "AzureUpdateDelivery", "category": "AzurePublic", "ip": "20.12.23.50:443", "subnet": "", "vnet": "", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "10.0.1.75:9", "name": "vm-build-agent-win-server-1", "category": "UnknownPrivate", "ip": "10.0.1.75:9", "subnet": "vms", "vnet": "vnet-management", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "10.0.1.75:80", "name": "vm-build-agent-win-server-1", "category": "InterVNet", "ip": "10.0.1.75:80", "subnet": "vms", "vnet": "vnet-management", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "192.166.1.4:80", "name": "vm-cwhd-s2s-onpremise", "category": "InterVNet", "ip": "192.166.1.4:80", "subnet": "vm", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "172.18.0.4:54321", "name": "managed vm in gatewaysubnet subnet", "category": "IntraVNet", "ip": "172.18.0.4:54321", "subnet": "gatewaysubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "192.166.0.68:5701", "name": "bastion-vm", "category": "IntraVNet", "ip": "192.166.0.68:5701", "subnet": "azurebastionsubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "192.166.0.4:54321", "name": "managed vm in gatewaysubnet subnet", "category": "IntraVNet", "ip": "192.166.0.4:54321", "subnet": "gatewaysubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "12.0.0.4:80", "name": "appgw-aks", "category": "InterVNet", "ip": "12.0.0.4:80", "subnet": "appgateway", "vnet": "vnet-aks", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "192.166.0.5:49756", "name": "managed vm in gatewaysubnet subnet", "category": "IntraVNet", "ip": "192.166.0.5:49756", "subnet": "gatewaysubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "172.18.0.69:443", "name": "bastion-vm", "category": "MaliciousFlow", "ip": "172.18.0.69:443", "subnet": "azurebastionsubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "52.163.205.184:500", "name": "vpngw-cwhd-s2s-azure-VM-52.163.205.184", "category": "AzurePublic", "ip": "52.163.205.184:500", "subnet": "gatewaysubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "10.0.1.5:23184", "name": "bastion-vm", "category": "AzurePublic", "ip": "10.0.1.5:23184", "subnet": "azurebastionsubnet", "vnet": "vnet-management", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "192.166.0.4:20195", "name": "vnetgw-cwhd-onpremise-VM-192.166.0.4", "category": "AzurePublic", "ip": "192.166.0.4:20195", "subnet": "gatewaysubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "172.18.1.4:21907", "name": "vm-cwhd-s2s-azure", "category": "AzurePublic", "ip": "172.18.1.4:21907", "subnet": "vm", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "172.16.2.5:29848", "name": "managed vm in apim subnet", "category": "AzurePublic", "ip": "172.16.2.5:29848", "subnet": "apim", "vnet": "vnet-ingress", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "172.18.0.5:29422", "name": "vpngw-cwhd-s2s-azure-VM-172.18.0.5", "category": "AzurePublic", "ip": "172.18.0.5:29422", "subnet": "gatewaysubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "172.18.0.4:25885", "name": "vpngw-cwhd-s2s-azure-VM-172.18.0.4", "category": "AzurePublic", "ip": "172.18.0.4:25885", "subnet": "gatewaysubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "192.166.0.5:22176", "name": "vnetgw-cwhd-onpremise-VM-192.166.0.5", "category": "AzurePublic", "ip": "192.166.0.5:22176", "subnet": "gatewaysubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "10.0.1.75:29975", "name": "vm-build-agent-win-server-1", "category": "AzurePublic", "ip": "10.0.1.75:29975", "subnet": "vms", "vnet": "vnet-management", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "172.16.2.5:3443", "name": "managed vm in apim subnet", "category": "AzurePublic", "ip": "172.16.2.5:3443", "subnet": "apim", "vnet": "vnet-ingress", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "192.166.0.4:8443", "name": "vnetgw-cwhd-onpremise-VM-192.166.0.4", "category": "AzurePublic", "ip": "192.166.0.4:8443", "subnet": "gatewaysubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "192.166.0.5:8444", "name": "vnetgw-cwhd-onpremise-VM-192.166.0.5", "category": "AzurePublic", "ip": "192.166.0.5:8444", "subnet": "gatewaysubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "172.18.0.4:8443", "name": "vpngw-cwhd-s2s-azure-VM-172.18.0.4", "category": "AzurePublic", "ip": "172.18.0.4:8443", "subnet": "gatewaysubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "172.18.0.5:8444", "name": "vpngw-cwhd-s2s-azure-VM-172.18.0.5", "category": "AzurePublic", "ip": "172.18.0.5:8444", "subnet": "gatewaysubnet", "vnet": "vnet-cwhd-s2s-azure", "srcPIPLocation": null, "destPIPLocation": null}, {"id": "52.187.163.69:500", "name": "vnetgw-cwhd-onpremise-VM-52.187.163.69", "category": "AzurePublic", "ip": "52.187.163.69:500", "subnet": "gatewaysubnet", "vnet": "vnet-cwh-s2s-onpremise", "srcPIPLocation": null, "destPIPLocation": null}], "edges": [{"source": "192.166.0.4", "target": "13.65.254.147:22215", "src_to_dest_data_size": "928 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "172.16.2.5", "target": "13.67.8.104:443", "src_to_dest_data_size": "32.42 MB", "dest_to_srct_data_size": "33.69 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "13.71.196.56:1886", "src_to_dest_data_size": "592.33 KB", "dest_to_srct_data_size": "253.69 KB", "flowType": "AzurePublic", "protocol": "tcp:leoip", "connectionType": ""}, {"source": "172.18.0.5", "target": "13.71.196.56:1886", "src_to_dest_data_size": "570.76 KB", "dest_to_srct_data_size": "193.54 KB", "flowType": "AzurePublic", "protocol": "tcp:leoip", "connectionType": ""}, {"source": "192.166.0.4", "target": "13.71.196.56:1886", "src_to_dest_data_size": "493.9 KB", "dest_to_srct_data_size": "253.59 KB", "flowType": "AzurePublic", "protocol": "tcp:leoip", "connectionType": ""}, {"source": "172.18.0.4", "target": "13.71.196.56:443", "src_to_dest_data_size": "1.37 MB", "dest_to_srct_data_size": "489.8 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "13.71.196.56:1886", "src_to_dest_data_size": "503.11 KB", "dest_to_srct_data_size": "273.88 KB", "flowType": "AzurePublic", "protocol": "tcp:leoip", "connectionType": ""}, {"source": "172.18.0.5", "target": "13.71.196.56:443", "src_to_dest_data_size": "1.2 MB", "dest_to_srct_data_size": "475.5 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "13.71.196.56:443", "src_to_dest_data_size": "1.12 MB", "dest_to_srct_data_size": "424.65 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "13.86.223.176:12000", "src_to_dest_data_size": "6.71 KB", "dest_to_srct_data_size": "2.08 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "13.86.223.176:12000", "src_to_dest_data_size": "3.51 KB", "dest_to_srct_data_size": "425 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "13.86.223.176:12000", "src_to_dest_data_size": "5.9 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.150.28.164:443", "src_to_dest_data_size": "7.06 MB", "dest_to_srct_data_size": "20.02 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.189.173.1:443", "src_to_dest_data_size": "2.89 KB", "dest_to_srct_data_size": "5.53 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.195.82.160:443", "src_to_dest_data_size": "5.41 MB", "dest_to_srct_data_size": "17.35 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.195.82.160:443", "src_to_dest_data_size": "7.1 MB", "dest_to_srct_data_size": "20.46 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.195.82.188:443", "src_to_dest_data_size": "7.47 MB", "dest_to_srct_data_size": "20.49 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.195.82.188:443", "src_to_dest_data_size": "5.18 MB", "dest_to_srct_data_size": "17.13 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.209.110.8:443", "src_to_dest_data_size": "107.93 KB", "dest_to_srct_data_size": "161.12 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.209.110.8:443", "src_to_dest_data_size": "157.97 KB", "dest_to_srct_data_size": "231.93 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.60.105.43:443", "src_to_dest_data_size": "158.31 KB", "dest_to_srct_data_size": "224.85 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.60.105.43:443", "src_to_dest_data_size": "97.17 KB", "dest_to_srct_data_size": "131.55 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.60.136.97:443", "src_to_dest_data_size": "32.11 MB", "dest_to_srct_data_size": "33.4 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "52.154.220.73:443", "src_to_dest_data_size": "2.31 MB", "dest_to_srct_data_size": "1.26 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "52.154.220.73:443", "src_to_dest_data_size": "1.8 MB", "dest_to_srct_data_size": "681.78 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.154.220.73:443", "src_to_dest_data_size": "1.08 MB", "dest_to_srct_data_size": "1.26 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "13.69.109.238:12000", "src_to_dest_data_size": "6.57 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.5", "target": "13.69.109.238:12000", "src_to_dest_data_size": "4.45 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "13.69.109.238:12000", "src_to_dest_data_size": "4.83 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "13.69.109.238:12000", "src_to_dest_data_size": "3.54 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "13.69.109.242:12000", "src_to_dest_data_size": "5.16 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.5", "target": "13.69.109.242:12000", "src_to_dest_data_size": "6.12 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "13.69.109.242:12000", "src_to_dest_data_size": "3.54 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.5", "target": "13.86.223.177:12000", "src_to_dest_data_size": "6.03 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.5", "target": "13.86.223.177:12000", "src_to_dest_data_size": "6.87 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "10.0.1.5", "target": "20.150.28.228:443", "src_to_dest_data_size": "11.15 MB", "dest_to_srct_data_size": "9.86 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.68", "target": "20.150.28.228:443", "src_to_dest_data_size": "8.02 MB", "dest_to_srct_data_size": "9.88 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.4", "target": "20.150.86.4:443", "src_to_dest_data_size": "11.37 MB", "dest_to_srct_data_size": "10.47 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.68", "target": "20.150.86.4:443", "src_to_dest_data_size": "9.42 MB", "dest_to_srct_data_size": "9.88 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.150.86.6:443", "src_to_dest_data_size": "7.32 MB", "dest_to_srct_data_size": "20.52 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.150.86.6:443", "src_to_dest_data_size": "7.08 MB", "dest_to_srct_data_size": "20.2 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.190.163.18:443", "src_to_dest_data_size": "7.08 MB", "dest_to_srct_data_size": "20.33 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.44.11.209:12000", "src_to_dest_data_size": "5.91 KB", "dest_to_srct_data_size": "3.32 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.44.11.209:12000", "src_to_dest_data_size": "4.77 KB", "dest_to_srct_data_size": "4.35 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.44.11.209:12000", "src_to_dest_data_size": "3.13 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.44.11.209:12000", "src_to_dest_data_size": "3.88 KB", "dest_to_srct_data_size": "2.49 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.51.7.8:12000", "src_to_dest_data_size": "3.22 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.51.7.8:12000", "src_to_dest_data_size": "1.61 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.51.7.8:12000", "src_to_dest_data_size": "3.54 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.51.7.8:12000", "src_to_dest_data_size": "3.54 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.51.7.8:12000", "src_to_dest_data_size": "2.26 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.60.136.102:445", "src_to_dest_data_size": "123.32 KB", "dest_to_srct_data_size": "90.66 KB", "flowType": "AzurePublic", "protocol": "tcp:microsoft-ds", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.60.81.109:443", "src_to_dest_data_size": "136.55 KB", "dest_to_srct_data_size": "246.23 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.60.81.109:443", "src_to_dest_data_size": "34.15 KB", "dest_to_srct_data_size": "107.98 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.69", "target": "23.98.114.99:443", "src_to_dest_data_size": "6.53 MB", "dest_to_srct_data_size": "9.82 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.68", "target": "23.98.114.99:443", "src_to_dest_data_size": "5.23 MB", "dest_to_srct_data_size": "3.26 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "40.117.147.74:443", "src_to_dest_data_size": "278.38 KB", "dest_to_srct_data_size": "467.83 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "40.117.147.74:443", "src_to_dest_data_size": "283.42 KB", "dest_to_srct_data_size": "473.7 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "40.117.147.74:443", "src_to_dest_data_size": "532.67 KB", "dest_to_srct_data_size": "878.6 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "40.117.147.74:443", "src_to_dest_data_size": "503.62 KB", "dest_to_srct_data_size": "809.8 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "40.78.247.96:12000", "src_to_dest_data_size": "2.51 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.5", "target": "40.78.247.96:12000", "src_to_dest_data_size": "1.29 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.5", "target": "40.78.247.96:12000", "src_to_dest_data_size": "1.61 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "40.78.247.96:12000", "src_to_dest_data_size": "1.55 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "13.107.42.16:443", "src_to_dest_data_size": "2.85 KB", "dest_to_srct_data_size": "8.44 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "13.107.42.16:443", "src_to_dest_data_size": "2.85 KB", "dest_to_srct_data_size": "8.57 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "13.107.42.16:443", "src_to_dest_data_size": "2.85 KB", "dest_to_srct_data_size": "8.57 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "13.86.223.179:12000", "src_to_dest_data_size": "8.96 KB", "dest_to_srct_data_size": "1.66 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "13.86.223.179:12000", "src_to_dest_data_size": "7.26 KB", "dest_to_srct_data_size": "2.08 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "13.86.223.179:12000", "src_to_dest_data_size": "6.48 KB", "dest_to_srct_data_size": "2.08 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "13.86.223.179:12000", "src_to_dest_data_size": "17.42 KB", "dest_to_srct_data_size": "2.17 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "13.92.40.223:443", "src_to_dest_data_size": "1.83 MB", "dest_to_srct_data_size": "2.61 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "13.92.40.223:1886", "src_to_dest_data_size": "82.3 KB", "dest_to_srct_data_size": "88.03 KB", "flowType": "AzurePublic", "protocol": "tcp:leoip", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.150.81.164:443", "src_to_dest_data_size": "1.33 MB", "dest_to_srct_data_size": "461.39 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.150.81.164:443", "src_to_dest_data_size": "1.2 MB", "dest_to_srct_data_size": "434.97 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.150.81.4:443", "src_to_dest_data_size": "1.2 MB", "dest_to_srct_data_size": "474.6 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.150.81.4:443", "src_to_dest_data_size": "18 KB", "dest_to_srct_data_size": "18.03 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.150.81.4:443", "src_to_dest_data_size": "1.31 MB", "dest_to_srct_data_size": "398.37 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.150.86.102:443", "src_to_dest_data_size": "7.19 MB", "dest_to_srct_data_size": "20.58 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.68", "target": "20.195.81.86:443", "src_to_dest_data_size": "9.3 MB", "dest_to_srct_data_size": "12.6 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.69", "target": "20.195.81.86:443", "src_to_dest_data_size": "7.63 MB", "dest_to_srct_data_size": "10.12 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.68", "target": "20.209.21.97:443", "src_to_dest_data_size": "5.47 MB", "dest_to_srct_data_size": "3.35 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.42.65.200:12000", "src_to_dest_data_size": "5.1 KB", "dest_to_srct_data_size": "2.91 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.42.65.200:12000", "src_to_dest_data_size": "6.35 KB", "dest_to_srct_data_size": "4.78 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.42.65.200:12000", "src_to_dest_data_size": "5.53 KB", "dest_to_srct_data_size": "3.74 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.42.65.200:12000", "src_to_dest_data_size": "2.97 KB", "dest_to_srct_data_size": "1.66 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.42.65.202:12000", "src_to_dest_data_size": "5.21 KB", "dest_to_srct_data_size": "3.74 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.42.65.202:12000", "src_to_dest_data_size": "4.91 KB", "dest_to_srct_data_size": "3.32 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.42.65.202:12000", "src_to_dest_data_size": "2.2 KB", "dest_to_srct_data_size": "2.17 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.42.65.202:12000", "src_to_dest_data_size": "3.98 KB", "dest_to_srct_data_size": "2.91 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.42.73.26:443", "src_to_dest_data_size": "241.95 KB", "dest_to_srct_data_size": "73 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.44.239.154:443", "src_to_dest_data_size": "5.47 MB", "dest_to_srct_data_size": "17.52 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.49.93.224:12000", "src_to_dest_data_size": "4.74 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.49.93.224:12000", "src_to_dest_data_size": "4.8 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.49.93.224:12000", "src_to_dest_data_size": "660 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.49.93.224:12000", "src_to_dest_data_size": "2.9 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "23.15.147.118:80", "src_to_dest_data_size": "1.18 KB", "dest_to_srct_data_size": "1.51 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.5", "target": "23.210.96.161:80", "src_to_dest_data_size": "590 Bytes", "dest_to_srct_data_size": "869 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "23.210.96.161:80", "src_to_dest_data_size": "1.57 KB", "dest_to_srct_data_size": "2.06 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.68", "target": "40.77.24.27:443", "src_to_dest_data_size": "5.61 KB", "dest_to_srct_data_size": "12.5 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.69", "target": "40.77.24.27:443", "src_to_dest_data_size": "5.61 KB", "dest_to_srct_data_size": "12.5 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.4", "target": "40.77.24.27:443", "src_to_dest_data_size": "5.84 KB", "dest_to_srct_data_size": "12.6 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "40.77.24.27:443", "src_to_dest_data_size": "942.26 KB", "dest_to_srct_data_size": "1.24 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.69", "target": "40.77.24.27:443", "src_to_dest_data_size": "5.71 KB", "dest_to_srct_data_size": "12.49 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.5", "target": "40.77.24.27:443", "src_to_dest_data_size": "798.84 KB", "dest_to_srct_data_size": "25.39 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "40.77.24.27:443", "src_to_dest_data_size": "1.85 MB", "dest_to_srct_data_size": "696.46 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.68", "target": "40.77.24.27:443", "src_to_dest_data_size": "5.64 KB", "dest_to_srct_data_size": "12.5 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "40.77.24.27:443", "src_to_dest_data_size": "350.83 KB", "dest_to_srct_data_size": "650.54 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "40.77.24.27:443", "src_to_dest_data_size": "696.26 KB", "dest_to_srct_data_size": "1.24 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "40.78.243.16:1886", "src_to_dest_data_size": "510.19 KB", "dest_to_srct_data_size": "266.77 KB", "flowType": "AzurePublic", "protocol": "tcp:leoip", "connectionType": ""}, {"source": "192.166.0.5", "target": "40.78.243.16:1886", "src_to_dest_data_size": "592.19 KB", "dest_to_srct_data_size": "253.66 KB", "flowType": "AzurePublic", "protocol": "tcp:leoip", "connectionType": ""}, {"source": "192.166.0.5", "target": "40.78.243.16:443", "src_to_dest_data_size": "1.23 MB", "dest_to_srct_data_size": "616.52 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "40.78.243.16:443", "src_to_dest_data_size": "1.43 MB", "dest_to_srct_data_size": "673.51 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "40.78.243.16:443", "src_to_dest_data_size": "1.4 MB", "dest_to_srct_data_size": "631.31 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "40.78.243.16:1886", "src_to_dest_data_size": "524.59 KB", "dest_to_srct_data_size": "275.81 KB", "flowType": "AzurePublic", "protocol": "tcp:leoip", "connectionType": ""}, {"source": "172.18.0.5", "target": "40.78.243.16:443", "src_to_dest_data_size": "1.28 MB", "dest_to_srct_data_size": "685.86 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "40.78.243.16:1886", "src_to_dest_data_size": "437.21 KB", "dest_to_srct_data_size": "54.49 KB", "flowType": "AzurePublic", "protocol": "tcp:leoip", "connectionType": ""}, {"source": "192.166.0.4", "target": "52.148.114.188:123", "src_to_dest_data_size": "5.01 KB", "dest_to_srct_data_size": "7.35 KB", "flowType": "AzurePublic", "protocol": "udp:ntp", "connectionType": ""}, {"source": "172.18.1.4", "target": "52.148.114.188:123", "src_to_dest_data_size": "5.19 KB", "dest_to_srct_data_size": "7.61 KB", "flowType": "AzurePublic", "protocol": "udp:ntp", "connectionType": ""}, {"source": "192.166.0.5", "target": "52.148.114.188:123", "src_to_dest_data_size": "5.01 KB", "dest_to_srct_data_size": "7.35 KB", "flowType": "AzurePublic", "protocol": "udp:ntp", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.148.114.188:123", "src_to_dest_data_size": "4.92 KB", "dest_to_srct_data_size": "7.22 KB", "flowType": "AzurePublic", "protocol": "udp:ntp", "connectionType": ""}, {"source": "10.0.1.75", "target": "52.148.114.188:123", "src_to_dest_data_size": "5.01 KB", "dest_to_srct_data_size": "7.35 KB", "flowType": "AzurePublic", "protocol": "udp:ntp", "connectionType": ""}, {"source": "172.18.0.4", "target": "52.148.114.188:123", "src_to_dest_data_size": "5.01 KB", "dest_to_srct_data_size": "7.35 KB", "flowType": "AzurePublic", "protocol": "udp:ntp", "connectionType": ""}, {"source": "172.18.0.5", "target": "52.148.114.188:123", "src_to_dest_data_size": "5.01 KB", "dest_to_srct_data_size": "7.35 KB", "flowType": "AzurePublic", "protocol": "udp:ntp", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.167.145.162:12000", "src_to_dest_data_size": "8.55 KB", "dest_to_srct_data_size": "1.3 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "10.0.1.4", "target": "52.239.197.68:443", "src_to_dest_data_size": "9.45 MB", "dest_to_srct_data_size": "10.13 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "13.69.109.239:12000", "src_to_dest_data_size": "7.03 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.5", "target": "13.69.109.239:12000", "src_to_dest_data_size": "4.83 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.150.81.36:443", "src_to_dest_data_size": "1.34 MB", "dest_to_srct_data_size": "461.88 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.150.81.36:443", "src_to_dest_data_size": "1.21 MB", "dest_to_srct_data_size": "475.72 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.150.81.36:443", "src_to_dest_data_size": "18 KB", "dest_to_srct_data_size": "17.94 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.150.81.36:443", "src_to_dest_data_size": "1.23 MB", "dest_to_srct_data_size": "503.12 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.190.144.162:443", "src_to_dest_data_size": "8.05 KB", "dest_to_srct_data_size": "29.56 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.209.15.66:443", "src_to_dest_data_size": "161.47 KB", "dest_to_srct_data_size": "208.13 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.209.15.66:443", "src_to_dest_data_size": "103.33 KB", "dest_to_srct_data_size": "136.9 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "20.42.73.25:443", "src_to_dest_data_size": "227.13 KB", "dest_to_srct_data_size": "117.28 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.42.73.25:443", "src_to_dest_data_size": "273.09 KB", "dest_to_srct_data_size": "457.41 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.42.73.25:443", "src_to_dest_data_size": "519.68 KB", "dest_to_srct_data_size": "844.1 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.44.11.245:12000", "src_to_dest_data_size": "3.81 KB", "dest_to_srct_data_size": "2.49 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.44.11.245:12000", "src_to_dest_data_size": "5.16 KB", "dest_to_srct_data_size": "2.49 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.44.11.245:12000", "src_to_dest_data_size": "3.49 KB", "dest_to_srct_data_size": "2.17 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.44.11.245:12000", "src_to_dest_data_size": "4.94 KB", "dest_to_srct_data_size": "2.91 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.44.11.245:12000", "src_to_dest_data_size": "2.43 KB", "dest_to_srct_data_size": "2.08 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.60.118.11:443", "src_to_dest_data_size": "107.36 KB", "dest_to_srct_data_size": "144.67 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.60.118.11:443", "src_to_dest_data_size": "155.49 KB", "dest_to_srct_data_size": "211 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "23.98.114.96:443", "src_to_dest_data_size": "5.63 MB", "dest_to_srct_data_size": "17.3 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.68", "target": "23.98.114.96:443", "src_to_dest_data_size": "8.64 MB", "dest_to_srct_data_size": "9.96 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "40.78.247.68:12000", "src_to_dest_data_size": "10.05 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "40.79.170.241:9201", "src_to_dest_data_size": "30.11 KB", "dest_to_srct_data_size": "19.53 KB", "flowType": "AzurePublic", "protocol": "tcp:wap-wsp-wtp", "connectionType": ""}, {"source": "172.18.0.4", "target": "40.79.170.241:9201", "src_to_dest_data_size": "30.16 KB", "dest_to_srct_data_size": "18.92 KB", "flowType": "AzurePublic", "protocol": "tcp:wap-wsp-wtp", "connectionType": ""}, {"source": "192.166.0.4", "target": "40.79.170.241:443", "src_to_dest_data_size": "6.63 KB", "dest_to_srct_data_size": "31.99 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "40.79.170.241:443", "src_to_dest_data_size": "6.66 KB", "dest_to_srct_data_size": "32 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "52.167.249.196:443", "src_to_dest_data_size": "2.28 KB", "dest_to_srct_data_size": "5.39 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.183.41.109:1886", "src_to_dest_data_size": "86.13 KB", "dest_to_srct_data_size": "91.14 KB", "flowType": "AzurePublic", "protocol": "tcp:leoip", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.183.41.109:443", "src_to_dest_data_size": "1.09 MB", "dest_to_srct_data_size": "1.74 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "52.191.219.104:443", "src_to_dest_data_size": "253.66 KB", "dest_to_srct_data_size": "416.35 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.68", "target": "13.86.218.252:443", "src_to_dest_data_size": "264.45 KB", "dest_to_srct_data_size": "249.05 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.5", "target": "13.86.218.252:443", "src_to_dest_data_size": "268.77 KB", "dest_to_srct_data_size": "253.83 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.69", "target": "13.86.218.252:443", "src_to_dest_data_size": "262.21 KB", "dest_to_srct_data_size": "248.55 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.4", "target": "13.86.218.252:443", "src_to_dest_data_size": "268.65 KB", "dest_to_srct_data_size": "253.63 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.68", "target": "13.86.218.252:443", "src_to_dest_data_size": "264.01 KB", "dest_to_srct_data_size": "247.42 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.69", "target": "13.86.218.252:443", "src_to_dest_data_size": "269.65 KB", "dest_to_srct_data_size": "254.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "146.75.46.172:80", "src_to_dest_data_size": "1.21 KB", "dest_to_srct_data_size": "1.39 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.4", "target": "146.75.46.172:80", "src_to_dest_data_size": "623 Bytes", "dest_to_srct_data_size": "712 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.4", "target": "146.75.46.172:80", "src_to_dest_data_size": "1.21 KB", "dest_to_srct_data_size": "1.39 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "146.75.46.172:80", "src_to_dest_data_size": "1.54 KB", "dest_to_srct_data_size": "1.78 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "146.75.46.172:80", "src_to_dest_data_size": "647 Bytes", "dest_to_srct_data_size": "712 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "10.0.1.75", "target": "146.75.46.172:80", "src_to_dest_data_size": "13.99 KB", "dest_to_srct_data_size": "318.75 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.5", "target": "146.75.46.172:80", "src_to_dest_data_size": "1.21 KB", "dest_to_srct_data_size": "1.39 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.195.65.39:1433", "src_to_dest_data_size": "2.86 MB", "dest_to_srct_data_size": "25.61 MB", "flowType": "AzurePublic", "protocol": "tcp:ms-sql-s", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.42.65.201:12000", "src_to_dest_data_size": "3.69 KB", "dest_to_srct_data_size": "2.49 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.42.65.201:12000", "src_to_dest_data_size": "5.3 KB", "dest_to_srct_data_size": "3.32 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.60.80.43:443", "src_to_dest_data_size": "163.8 KB", "dest_to_srct_data_size": "216.68 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.5", "target": "23.98.114.97:443", "src_to_dest_data_size": "11.14 MB", "dest_to_srct_data_size": "10.02 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "52.137.106.217:443", "src_to_dest_data_size": "1.76 KB", "dest_to_srct_data_size": "5.13 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "52.137.106.217:443", "src_to_dest_data_size": "1.72 KB", "dest_to_srct_data_size": "5.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.165.27.187:443", "src_to_dest_data_size": "880.12 KB", "dest_to_srct_data_size": "1.22 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "52.165.27.187:443", "src_to_dest_data_size": "1016.64 KB", "dest_to_srct_data_size": "1.29 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.168.117.168:443", "src_to_dest_data_size": "1.72 MB", "dest_to_srct_data_size": "2.4 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.5", "target": "20.209.117.229:443", "src_to_dest_data_size": "1.25 MB", "dest_to_srct_data_size": "28.82 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.4", "target": "20.209.117.229:443", "src_to_dest_data_size": "1.25 MB", "dest_to_srct_data_size": "28.93 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.69", "target": "20.209.117.229:443", "src_to_dest_data_size": "1.24 MB", "dest_to_srct_data_size": "15.85 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.68", "target": "20.60.195.131:443", "src_to_dest_data_size": "806.5 KB", "dest_to_srct_data_size": "67.72 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.4", "target": "20.60.195.131:443", "src_to_dest_data_size": "796.86 KB", "dest_to_srct_data_size": "31.51 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.60.195.131:443", "src_to_dest_data_size": "629.53 KB", "dest_to_srct_data_size": "622.72 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.60.195.131:443", "src_to_dest_data_size": "962.81 KB", "dest_to_srct_data_size": "1.23 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.189.173.4:443", "src_to_dest_data_size": "84.42 KB", "dest_to_srct_data_size": "155.38 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.189.173.4:443", "src_to_dest_data_size": "2.94 KB", "dest_to_srct_data_size": "5.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.4", "target": "13.107.246.59:443", "src_to_dest_data_size": "52.94 KB", "dest_to_srct_data_size": "4.08 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.5", "target": "13.107.246.59:443", "src_to_dest_data_size": "775.25 KB", "dest_to_srct_data_size": "176.26 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "13.107.246.59:443", "src_to_dest_data_size": "1.39 KB", "dest_to_srct_data_size": "10.17 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "13.69.233.97:12000", "src_to_dest_data_size": "2.23 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.5", "target": "13.69.233.97:12000", "src_to_dest_data_size": "3.45 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "13.69.233.97:12000", "src_to_dest_data_size": "3.83 KB", "dest_to_srct_data_size": "1.74 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.5", "target": "13.69.233.97:12000", "src_to_dest_data_size": "3.13 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "13.69.233.97:12000", "src_to_dest_data_size": "1.61 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.150.111.68:443", "src_to_dest_data_size": "111.81 KB", "dest_to_srct_data_size": "144.55 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.150.111.68:443", "src_to_dest_data_size": "156.88 KB", "dest_to_srct_data_size": "207.75 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.195.65.33:1433", "src_to_dest_data_size": "2.85 MB", "dest_to_srct_data_size": "25.5 MB", "flowType": "AzurePublic", "protocol": "tcp:ms-sql-s", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.49.93.221:12000", "src_to_dest_data_size": "5.36 KB", "dest_to_srct_data_size": "1.66 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.49.93.221:12000", "src_to_dest_data_size": "3.85 KB", "dest_to_srct_data_size": "1.3 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.60.4.133:443", "src_to_dest_data_size": "1.33 MB", "dest_to_srct_data_size": "462.19 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.60.4.133:443", "src_to_dest_data_size": "18.11 KB", "dest_to_srct_data_size": "18.03 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.60.4.133:443", "src_to_dest_data_size": "1.22 MB", "dest_to_srct_data_size": "449.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.69", "target": "23.98.114.98:443", "src_to_dest_data_size": "8.62 MB", "dest_to_srct_data_size": "10.11 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "23.98.114.98:443", "src_to_dest_data_size": "5.59 MB", "dest_to_srct_data_size": "17.5 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.69", "target": "23.98.114.98:443", "src_to_dest_data_size": "8.33 MB", "dest_to_srct_data_size": "9.76 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.69", "target": "40.71.13.173:443", "src_to_dest_data_size": "311.43 KB", "dest_to_srct_data_size": "315.9 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.4", "target": "40.71.13.173:443", "src_to_dest_data_size": "304.54 KB", "dest_to_srct_data_size": "302.45 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.69", "target": "40.71.13.173:443", "src_to_dest_data_size": "301.98 KB", "dest_to_srct_data_size": "302.67 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.68", "target": "40.71.13.173:443", "src_to_dest_data_size": "299.94 KB", "dest_to_srct_data_size": "298.35 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.68", "target": "40.71.13.173:443", "src_to_dest_data_size": "304.99 KB", "dest_to_srct_data_size": "304.5 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.5", "target": "40.71.13.173:443", "src_to_dest_data_size": "302.57 KB", "dest_to_srct_data_size": "302.88 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.106.86.13:443", "src_to_dest_data_size": "1.31 KB", "dest_to_srct_data_size": "5.38 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.106.86.13:443", "src_to_dest_data_size": "4.23 KB", "dest_to_srct_data_size": "10.78 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.5", "target": "20.150.86.36:443", "src_to_dest_data_size": "12.01 MB", "dest_to_srct_data_size": "10.39 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.189.172.18:443", "src_to_dest_data_size": "464.97 KB", "dest_to_srct_data_size": "602.84 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.195.82.161:443", "src_to_dest_data_size": "7.4 MB", "dest_to_srct_data_size": "20.76 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.198.162.78:443", "src_to_dest_data_size": "1.24 KB", "dest_to_srct_data_size": "2 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.49.93.201:12000", "src_to_dest_data_size": "4.8 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.49.93.201:12000", "src_to_dest_data_size": "4.49 KB", "dest_to_srct_data_size": "890 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.49.93.201:12000", "src_to_dest_data_size": "2.58 KB", "dest_to_srct_data_size": "1.25 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.49.93.201:12000", "src_to_dest_data_size": "6.77 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.4", "target": "204.79.197.203:80", "src_to_dest_data_size": "750 Bytes", "dest_to_srct_data_size": "3.82 KB", "flowType": "AzurePublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "204.79.197.203:80", "src_to_dest_data_size": "750 Bytes", "dest_to_srct_data_size": "3.82 KB", "flowType": "AzurePublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.167.17.97:443", "src_to_dest_data_size": "2.29 KB", "dest_to_srct_data_size": "5.29 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "52.182.143.203:12000", "src_to_dest_data_size": "3.72 KB", "dest_to_srct_data_size": "2.08 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "192.166.0.5", "target": "52.182.143.203:12000", "src_to_dest_data_size": "5.36 KB", "dest_to_srct_data_size": "2.08 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.5", "target": "52.182.143.203:12000", "src_to_dest_data_size": "2.78 KB", "dest_to_srct_data_size": "1.66 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.182.143.203:12000", "src_to_dest_data_size": "5 KB", "dest_to_srct_data_size": "3.91 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.0.4", "target": "52.182.143.203:12000", "src_to_dest_data_size": "3.3 KB", "dest_to_srct_data_size": "3.32 KB", "flowType": "AzurePublic", "protocol": "tcp:entextxid", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.189.173.14:443", "src_to_dest_data_size": "61.08 KB", "dest_to_srct_data_size": "84.19 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.189.173.14:443", "src_to_dest_data_size": "2.94 KB", "dest_to_srct_data_size": "5.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.190.148.164:443", "src_to_dest_data_size": "3.1 KB", "dest_to_srct_data_size": "19.94 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "40.126.16.164:443", "src_to_dest_data_size": "2.03 KB", "dest_to_srct_data_size": "13.3 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "40.79.173.41:443", "src_to_dest_data_size": "9.47 KB", "dest_to_srct_data_size": "5.83 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "40.79.197.34:443", "src_to_dest_data_size": "2.89 KB", "dest_to_srct_data_size": "5.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.189.173.25:443", "src_to_dest_data_size": "95.32 KB", "dest_to_srct_data_size": "85.12 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.242.39.171:443", "src_to_dest_data_size": "247.67 KB", "dest_to_srct_data_size": "108.95 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.42.73.31:443", "src_to_dest_data_size": "539.12 KB", "dest_to_srct_data_size": "889.79 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "52.168.117.175:443", "src_to_dest_data_size": "529.33 KB", "dest_to_srct_data_size": "869.67 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "52.182.143.211:443", "src_to_dest_data_size": "137.7 KB", "dest_to_srct_data_size": "71.18 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "52.185.211.133:443", "src_to_dest_data_size": "2.18 KB", "dest_to_srct_data_size": "5.37 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.185.211.133:443", "src_to_dest_data_size": "4.67 KB", "dest_to_srct_data_size": "16.25 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "52.185.211.133:443", "src_to_dest_data_size": "1.64 KB", "dest_to_srct_data_size": "5.38 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "52.185.211.133:443", "src_to_dest_data_size": "2.37 KB", "dest_to_srct_data_size": "5.4 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.69", "target": "13.107.253.59:443", "src_to_dest_data_size": "271.24 KB", "dest_to_srct_data_size": "67.06 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "13.107.253.59:443", "src_to_dest_data_size": "1.42 KB", "dest_to_srct_data_size": "10.17 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "13.95.31.18:443", "src_to_dest_data_size": "104.43 KB", "dest_to_srct_data_size": "8.72 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.190.148.166:443", "src_to_dest_data_size": "4.17 KB", "dest_to_srct_data_size": "26.6 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.42.65.93:443", "src_to_dest_data_size": "1.77 MB", "dest_to_srct_data_size": "2.51 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "20.42.65.93:443", "src_to_dest_data_size": "286.2 KB", "dest_to_srct_data_size": "132.53 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.42.65.93:443", "src_to_dest_data_size": "159.4 KB", "dest_to_srct_data_size": "71.52 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "52.182.143.209:443", "src_to_dest_data_size": "82.28 KB", "dest_to_srct_data_size": "36.18 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "52.182.143.209:443", "src_to_dest_data_size": "99.22 KB", "dest_to_srct_data_size": "47.04 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.189.173.2:443", "src_to_dest_data_size": "371.5 KB", "dest_to_srct_data_size": "160.15 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "20.189.173.2:443", "src_to_dest_data_size": "2.94 KB", "dest_to_srct_data_size": "5.63 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "13.71.55.58:443", "src_to_dest_data_size": "1.7 KB", "dest_to_srct_data_size": "5.37 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "13.71.55.58:443", "src_to_dest_data_size": "1.64 KB", "dest_to_srct_data_size": "5.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "13.71.55.58:443", "src_to_dest_data_size": "1.46 KB", "dest_to_srct_data_size": "5.38 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "13.71.55.58:443", "src_to_dest_data_size": "1.64 KB", "dest_to_srct_data_size": "5.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.42.73.30:443", "src_to_dest_data_size": "531.78 KB", "dest_to_srct_data_size": "847.26 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.42.73.30:443", "src_to_dest_data_size": "181.91 KB", "dest_to_srct_data_size": "94.17 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.49.150.241:443", "src_to_dest_data_size": "2.18 KB", "dest_to_srct_data_size": "5.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "20.49.150.241:443", "src_to_dest_data_size": "2.43 KB", "dest_to_srct_data_size": "5.4 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "23.36.48.35:443", "src_to_dest_data_size": "2.71 KB", "dest_to_srct_data_size": "16.01 KB", "flowType": "ExternalPublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.42.73.27:443", "src_to_dest_data_size": "537.81 KB", "dest_to_srct_data_size": "884.02 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "20.42.73.27:443", "src_to_dest_data_size": "199.17 KB", "dest_to_srct_data_size": "122.83 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "52.140.118.28:443", "src_to_dest_data_size": "1.64 KB", "dest_to_srct_data_size": "5.13 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "52.140.118.28:443", "src_to_dest_data_size": "1.39 KB", "dest_to_srct_data_size": "5.25 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "52.140.118.28:443", "src_to_dest_data_size": "1.68 KB", "dest_to_srct_data_size": "5.21 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "104.208.16.91:443", "src_to_dest_data_size": "357.88 KB", "dest_to_srct_data_size": "657.19 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "104.46.162.227:443", "src_to_dest_data_size": "2.94 KB", "dest_to_srct_data_size": "5.66 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "104.46.162.227:443", "src_to_dest_data_size": "10.37 KB", "dest_to_srct_data_size": "6.03 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "52.178.17.235:443", "src_to_dest_data_size": "2.89 KB", "dest_to_srct_data_size": "5.86 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "104.208.16.88:443", "src_to_dest_data_size": "321.6 KB", "dest_to_srct_data_size": "86.87 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "23.210.250.120:80", "src_to_dest_data_size": "1.32 KB", "dest_to_srct_data_size": "1.57 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "51.105.71.137:443", "src_to_dest_data_size": "2.02 KB", "dest_to_srct_data_size": "5.68 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "51.11.168.232:443", "src_to_dest_data_size": "2.34 KB", "dest_to_srct_data_size": "5.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "51.11.168.232:443", "src_to_dest_data_size": "4.09 KB", "dest_to_srct_data_size": "271.09 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "51.11.168.232:443", "src_to_dest_data_size": "2.29 KB", "dest_to_srct_data_size": "5.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "51.11.168.232:443", "src_to_dest_data_size": "2.8 KB", "dest_to_srct_data_size": "5.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "104.46.162.225:443", "src_to_dest_data_size": "3.72 KB", "dest_to_srct_data_size": "5.63 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "51.105.71.136:443", "src_to_dest_data_size": "3 KB", "dest_to_srct_data_size": "5.73 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "51.105.71.136:443", "src_to_dest_data_size": "2.02 KB", "dest_to_srct_data_size": "5.68 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "52.183.220.149:443", "src_to_dest_data_size": "2.8 KB", "dest_to_srct_data_size": "5.4 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "52.183.220.149:443", "src_to_dest_data_size": "1.46 KB", "dest_to_srct_data_size": "5.38 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "52.183.220.149:443", "src_to_dest_data_size": "2.4 KB", "dest_to_srct_data_size": "5.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "52.183.220.149:443", "src_to_dest_data_size": "4.11 KB", "dest_to_srct_data_size": "10.78 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "13.89.179.10:443", "src_to_dest_data_size": "114.06 KB", "dest_to_srct_data_size": "87.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "40.79.167.8:443", "src_to_dest_data_size": "10.53 KB", "dest_to_srct_data_size": "6.16 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "20.42.65.91:443", "src_to_dest_data_size": "218.87 KB", "dest_to_srct_data_size": "135.77 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.42.73.28:443", "src_to_dest_data_size": "525.81 KB", "dest_to_srct_data_size": "868.43 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "23.61.202.49:80", "src_to_dest_data_size": "1.21 KB", "dest_to_srct_data_size": "1.32 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "10.0.1.75", "target": "20.189.173.5:443", "src_to_dest_data_size": "183.3 KB", "dest_to_srct_data_size": "104.67 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.198.162.76:443", "src_to_dest_data_size": "6.74 KB", "dest_to_srct_data_size": "3.36 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "104.46.162.224:443", "src_to_dest_data_size": "2.93 KB", "dest_to_srct_data_size": "5.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "43.174.32.117:80", "src_to_dest_data_size": "1.15 KB", "dest_to_srct_data_size": "1.45 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "104.208.16.89:443", "src_to_dest_data_size": "686.11 KB", "dest_to_srct_data_size": "1.22 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "184.87.193.149:80", "src_to_dest_data_size": "618 Bytes", "dest_to_srct_data_size": "676 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "184.87.193.149:80", "src_to_dest_data_size": "623 Bytes", "dest_to_srct_data_size": "675 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.68", "target": "184.87.193.149:80", "src_to_dest_data_size": "699 Bytes", "dest_to_srct_data_size": "34.36 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.189.173.8:443", "src_to_dest_data_size": "160.77 KB", "dest_to_srct_data_size": "222.54 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "20.189.173.11:443", "src_to_dest_data_size": "213.24 KB", "dest_to_srct_data_size": "106.35 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.189.173.18:443", "src_to_dest_data_size": "60.55 KB", "dest_to_srct_data_size": "112.42 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "51.104.15.252:443", "src_to_dest_data_size": "2.88 KB", "dest_to_srct_data_size": "5.86 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "51.104.15.252:443", "src_to_dest_data_size": "10.86 KB", "dest_to_srct_data_size": "6.02 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "40.69.42.241:443", "src_to_dest_data_size": "208.65 KB", "dest_to_srct_data_size": "15.64 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "52.182.141.63:443", "src_to_dest_data_size": "124.62 KB", "dest_to_srct_data_size": "48.07 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "52.182.143.215:443", "src_to_dest_data_size": "89.16 KB", "dest_to_srct_data_size": "58.6 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.5", "target": "184.87.193.143:80", "src_to_dest_data_size": "633 Bytes", "dest_to_srct_data_size": "34.36 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "10.0.1.75", "target": "43.174.14.43:80", "src_to_dest_data_size": "14.16 KB", "dest_to_srct_data_size": "315.28 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.4", "target": "43.174.14.43:80", "src_to_dest_data_size": "569 Bytes", "dest_to_srct_data_size": "813 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.42.65.88:443", "src_to_dest_data_size": "82.15 KB", "dest_to_srct_data_size": "41.02 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "13.89.178.26:443", "src_to_dest_data_size": "87.67 KB", "dest_to_srct_data_size": "42.01 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "23.61.202.51:80", "src_to_dest_data_size": "2.39 KB", "dest_to_srct_data_size": "2.83 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "23.61.202.56:80", "src_to_dest_data_size": "1.76 KB", "dest_to_srct_data_size": "2.11 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.4", "target": "52.149.20.212:443", "src_to_dest_data_size": "1.36 MB", "dest_to_srct_data_size": "593.9 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.168.112.67:443", "src_to_dest_data_size": "1.71 MB", "dest_to_srct_data_size": "2.37 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "13.89.178.27:443", "src_to_dest_data_size": "34.48 KB", "dest_to_srct_data_size": "29.82 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.4", "target": "23.61.202.154:80", "src_to_dest_data_size": "633 Bytes", "dest_to_srct_data_size": "34.36 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "10.0.1.5", "target": "23.61.202.181:80", "src_to_dest_data_size": "831 Bytes", "dest_to_srct_data_size": "34.36 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.4", "target": "23.61.202.42:80", "src_to_dest_data_size": "1.87 KB", "dest_to_srct_data_size": "2.08 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "23.61.202.57:80", "src_to_dest_data_size": "647 Bytes", "dest_to_srct_data_size": "675 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "51.116.246.104:443", "src_to_dest_data_size": "2.02 KB", "dest_to_srct_data_size": "5.67 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "51.116.246.104:443", "src_to_dest_data_size": "3.06 KB", "dest_to_srct_data_size": "5.63 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.68", "target": "34.107.245.121:443", "src_to_dest_data_size": "3.16 KB", "dest_to_srct_data_size": "6.79 KB", "flowType": "ExternalPublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.68", "target": "34.107.245.121:443", "src_to_dest_data_size": "3.22 KB", "dest_to_srct_data_size": "6.8 KB", "flowType": "ExternalPublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "52.182.143.208:443", "src_to_dest_data_size": "125.72 KB", "dest_to_srct_data_size": "71.35 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "52.182.143.208:443", "src_to_dest_data_size": "945.92 KB", "dest_to_srct_data_size": "694.34 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "101.33.26.246:80", "src_to_dest_data_size": "17.6 KB", "dest_to_srct_data_size": "1020.42 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.44.10.123:443", "src_to_dest_data_size": "652.19 KB", "dest_to_srct_data_size": "682.67 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "40.126.16.163:443", "src_to_dest_data_size": "1.02 KB", "dest_to_srct_data_size": "6.65 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "13.69.239.78:443", "src_to_dest_data_size": "2.93 KB", "dest_to_srct_data_size": "5.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.42.65.85:443", "src_to_dest_data_size": "140.57 KB", "dest_to_srct_data_size": "126.55 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "40.79.189.59:443", "src_to_dest_data_size": "12.2 KB", "dest_to_srct_data_size": "6.03 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "52.252.28.242:443", "src_to_dest_data_size": "105.86 KB", "dest_to_srct_data_size": "153.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "13.69.109.131:443", "src_to_dest_data_size": "5.66 KB", "dest_to_srct_data_size": "11.26 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "43.174.14.44:80", "src_to_dest_data_size": "564 Bytes", "dest_to_srct_data_size": "814 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "51.132.193.104:443", "src_to_dest_data_size": "2.83 KB", "dest_to_srct_data_size": "5.67 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.50.201.204:443", "src_to_dest_data_size": "2.93 KB", "dest_to_srct_data_size": "5.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "20.97.190.222:443", "src_to_dest_data_size": "129.24 KB", "dest_to_srct_data_size": "186.82 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "23.61.202.61:80", "src_to_dest_data_size": "1.21 KB", "dest_to_srct_data_size": "1.32 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "52.168.112.66:443", "src_to_dest_data_size": "139.41 KB", "dest_to_srct_data_size": "204.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "52.168.112.66:443", "src_to_dest_data_size": "243.81 KB", "dest_to_srct_data_size": "394.18 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.16.2.5", "target": "4.151.228.221:443", "src_to_dest_data_size": "3.5 KB", "dest_to_srct_data_size": "5.84 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "4.151.228.221:443", "src_to_dest_data_size": "3.49 KB", "dest_to_srct_data_size": "5.75 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "13.69.109.130:443", "src_to_dest_data_size": "3.12 KB", "dest_to_srct_data_size": "5.63 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.10.127.193:443", "src_to_dest_data_size": "85.83 KB", "dest_to_srct_data_size": "299.95 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "23.54.155.168:80", "src_to_dest_data_size": "618 Bytes", "dest_to_srct_data_size": "676 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.5", "target": "23.54.155.168:80", "src_to_dest_data_size": "1.82 KB", "dest_to_srct_data_size": "2.08 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.4", "target": "23.54.155.168:80", "src_to_dest_data_size": "1.82 KB", "dest_to_srct_data_size": "1.98 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "10.0.1.75", "target": "23.98.113.115:443", "src_to_dest_data_size": "1.32 MB", "dest_to_srct_data_size": "1.54 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "23.209.46.95:80", "src_to_dest_data_size": "1.82 KB", "dest_to_srct_data_size": "2.08 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.5", "target": "23.61.202.50:80", "src_to_dest_data_size": "1.18 KB", "dest_to_srct_data_size": "1.51 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "10.0.1.75", "target": "23.98.82.210:443", "src_to_dest_data_size": "1.31 MB", "dest_to_srct_data_size": "1.56 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "20.195.81.84:5671", "src_to_dest_data_size": "217.3 KB", "dest_to_srct_data_size": "48.96 KB", "flowType": "AzurePublic", "protocol": "tcp:amqps", "connectionType": ""}, {"source": "172.18.0.4", "target": "20.195.81.84:5671", "src_to_dest_data_size": "219.13 KB", "dest_to_srct_data_size": "48.96 KB", "flowType": "AzurePublic", "protocol": "tcp:amqps", "connectionType": ""}, {"source": "172.18.0.4", "target": "40.79.141.152:443", "src_to_dest_data_size": "2.88 KB", "dest_to_srct_data_size": "5.86 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.10.127.192:443", "src_to_dest_data_size": "80.01 KB", "dest_to_srct_data_size": "94.01 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "20.10.127.192:443", "src_to_dest_data_size": "80.85 KB", "dest_to_srct_data_size": "293.56 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "23.36.50.36:443", "src_to_dest_data_size": "2.79 KB", "dest_to_srct_data_size": "16.01 KB", "flowType": "ExternalPublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "20.242.181.1:443", "src_to_dest_data_size": "162.57 KB", "dest_to_srct_data_size": "104.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "23.45.176.172:80", "src_to_dest_data_size": "623 Bytes", "dest_to_srct_data_size": "675 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "23.45.176.172:80", "src_to_dest_data_size": "618 Bytes", "dest_to_srct_data_size": "676 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.5", "target": "23.45.176.162:80", "src_to_dest_data_size": "1.54 KB", "dest_to_srct_data_size": "1.77 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "23.45.176.162:80", "src_to_dest_data_size": "623 Bytes", "dest_to_srct_data_size": "675 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "23.45.176.162:80", "src_to_dest_data_size": "647 Bytes", "dest_to_srct_data_size": "675 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "10.0.1.75", "target": "23.54.155.174:80", "src_to_dest_data_size": "1.39 KB", "dest_to_srct_data_size": "1.57 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "23.54.155.174:80", "src_to_dest_data_size": "618 Bytes", "dest_to_srct_data_size": "676 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "23.54.155.174:80", "src_to_dest_data_size": "647 Bytes", "dest_to_srct_data_size": "675 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.4", "target": "23.54.155.174:80", "src_to_dest_data_size": "1.82 KB", "dest_to_srct_data_size": "1.98 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.4", "target": "23.54.155.174:80", "src_to_dest_data_size": "618 Bytes", "dest_to_srct_data_size": "676 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "20.15.141.192:443", "src_to_dest_data_size": "102.14 KB", "dest_to_srct_data_size": "277.07 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "146.75.114.172:80", "src_to_dest_data_size": "1.54 KB", "dest_to_srct_data_size": "1.78 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "146.75.114.172:80", "src_to_dest_data_size": "8.01 KB", "dest_to_srct_data_size": "403.69 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "146.75.114.172:80", "src_to_dest_data_size": "1013 Bytes", "dest_to_srct_data_size": "1.09 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.4", "target": "146.75.114.172:80", "src_to_dest_data_size": "1.21 KB", "dest_to_srct_data_size": "1.39 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "146.75.114.172:80", "src_to_dest_data_size": "618 Bytes", "dest_to_srct_data_size": "712 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.4", "target": "146.75.114.172:80", "src_to_dest_data_size": "1.21 KB", "dest_to_srct_data_size": "1.39 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "10.0.1.75", "target": "146.75.114.172:80", "src_to_dest_data_size": "36.25 KB", "dest_to_srct_data_size": "636.16 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "23.209.46.91:80", "src_to_dest_data_size": "647 Bytes", "dest_to_srct_data_size": "777 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "23.209.46.91:80", "src_to_dest_data_size": "1.51 KB", "dest_to_srct_data_size": "2.1 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "10.0.1.75", "target": "13.69.239.77:443", "src_to_dest_data_size": "4.18 KB", "dest_to_srct_data_size": "5.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.5", "target": "23.61.202.38:80", "src_to_dest_data_size": "1.18 KB", "dest_to_srct_data_size": "1.51 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "20.190.144.136:443", "src_to_dest_data_size": "2.14 KB", "dest_to_srct_data_size": "13.3 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.4", "target": "23.54.155.173:80", "src_to_dest_data_size": "1.16 KB", "dest_to_srct_data_size": "1.48 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "43.175.139.73:80", "src_to_dest_data_size": "564 Bytes", "dest_to_srct_data_size": "814 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.4", "target": "151.101.110.172:80", "src_to_dest_data_size": "618 Bytes", "dest_to_srct_data_size": "709 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "151.101.110.172:80", "src_to_dest_data_size": "36.15 KB", "dest_to_srct_data_size": "636.02 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "151.101.110.172:80", "src_to_dest_data_size": "1.21 KB", "dest_to_srct_data_size": "1.39 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.16.2.5", "target": "52.123.252.3:443", "src_to_dest_data_size": "2.71 KB", "dest_to_srct_data_size": "6.37 KB", "flowType": "ExternalPublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "151.101.90.172:80", "src_to_dest_data_size": "618 Bytes", "dest_to_srct_data_size": "713 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "151.101.90.172:80", "src_to_dest_data_size": "623 Bytes", "dest_to_srct_data_size": "712 Bytes", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.12.23.50:443", "src_to_dest_data_size": "2.76 KB", "dest_to_srct_data_size": "7.82 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.4", "target": "40.79.189.58:443", "src_to_dest_data_size": "2.94 KB", "dest_to_srct_data_size": "5.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "192.166.0.5", "target": "40.83.235.53:1688", "src_to_dest_data_size": "914 Bytes", "dest_to_srct_data_size": "962 Bytes", "flowType": "AzurePublic", "protocol": "tcp:nsjtp-data", "connectionType": ""}, {"source": "10.0.1.75", "target": "23.209.46.92:80", "src_to_dest_data_size": "1.39 KB", "dest_to_srct_data_size": "1.67 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.1.4", "target": "40.69.75.33:443", "src_to_dest_data_size": "9.74 KB", "dest_to_srct_data_size": "9.99 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "52.149.43.87:443", "src_to_dest_data_size": "1.36 KB", "dest_to_srct_data_size": "3.96 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "51.11.192.49:443", "src_to_dest_data_size": "3.12 KB", "dest_to_srct_data_size": "5.63 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.75", "target": "23.54.155.165:80", "src_to_dest_data_size": "2.64 KB", "dest_to_srct_data_size": "713.01 KB", "flowType": "ExternalPublic", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.5", "target": "20.50.73.9:443", "src_to_dest_data_size": "2.94 KB", "dest_to_srct_data_size": "5.76 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "172.18.0.69", "target": "10.0.1.75:9", "src_to_dest_data_size": "14.53 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "UnknownPrivate", "protocol": "tcp:discard", "connectionType": ""}, {"source": "172.18.1.4", "target": "10.0.1.75:80", "src_to_dest_data_size": "51.64 KB", "dest_to_srct_data_size": "150.9 KB", "flowType": "InterVNet", "protocol": "tcp:http", "connectionType": "VNetPeering"}, {"source": "172.18.1.4", "target": "192.166.1.4:80", "src_to_dest_data_size": "80.16 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "InterVNet", "protocol": "tcp:http", "connectionType": "VpnGateway"}, {"source": "172.18.0.68", "target": "10.0.1.75:80", "src_to_dest_data_size": "102.19 KB", "dest_to_srct_data_size": "138.75 KB", "flowType": "UnknownPrivate", "protocol": "tcp:http", "connectionType": ""}, {"source": "172.18.0.5", "target": "172.18.0.4:54321", "src_to_dest_data_size": "213.36 KB", "dest_to_srct_data_size": "213.37 KB", "flowType": "IntraVNet", "protocol": "tcp", "connectionType": ""}, {"source": "192.166.0.69", "target": "192.166.0.68:5701", "src_to_dest_data_size": "3.45 MB", "dest_to_srct_data_size": "2.72 MB", "flowType": "IntraVNet", "protocol": "tcp", "connectionType": ""}, {"source": "192.166.0.5", "target": "192.166.0.4:54321", "src_to_dest_data_size": "230.1 KB", "dest_to_srct_data_size": "230.2 KB", "flowType": "IntraVNet", "protocol": "tcp", "connectionType": ""}, {"source": "10.0.1.5", "target": "10.0.1.4:5701", "src_to_dest_data_size": "2.52 MB", "dest_to_srct_data_size": "3.39 MB", "flowType": "IntraVNet", "protocol": "tcp", "connectionType": ""}, {"source": "10.0.1.75", "target": "12.0.0.4:80", "src_to_dest_data_size": "81.56 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "InterVNet", "protocol": "tcp:http", "connectionType": "VNetPeering"}, {"source": "172.18.0.69", "target": "172.18.0.68:5701", "src_to_dest_data_size": "3.24 MB", "dest_to_srct_data_size": "2.52 MB", "flowType": "IntraVNet", "protocol": "tcp", "connectionType": ""}, {"source": "172.18.0.68", "target": "10.0.1.75:9", "src_to_dest_data_size": "14.53 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "UnknownPrivate", "protocol": "tcp:discard", "connectionType": ""}, {"source": "172.18.0.69", "target": "10.0.1.75:80", "src_to_dest_data_size": "103.04 KB", "dest_to_srct_data_size": "139.91 KB", "flowType": "UnknownPrivate", "protocol": "tcp:http", "connectionType": ""}, {"source": "192.166.0.4", "target": "192.166.0.5:49756", "src_to_dest_data_size": "230.1 KB", "dest_to_srct_data_size": "230.3 KB", "flowType": "IntraVNet", "protocol": "tcp", "connectionType": ""}, {"source": "196.251.70.87", "target": "172.18.0.69:443", "src_to_dest_data_size": "2.47 KB", "dest_to_srct_data_size": "7.48 KB", "flowType": "MaliciousFlow", "protocol": "tcp:https", "connectionType": ""}, {"source": "10.0.1.4", "target": "10.0.1.5:5701", "src_to_dest_data_size": "283.84 KB", "dest_to_srct_data_size": "542.81 KB", "flowType": "IntraVNet", "protocol": "tcp", "connectionType": ""}, {"source": "52.187.163.69", "target": "52.163.205.184:500", "src_to_dest_data_size": "40.7 KB", "dest_to_srct_data_size": "30.08 KB", "flowType": "AzurePublic", "protocol": "udp:isakmp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23184", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24815", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23704", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20195", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:21907", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23207", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20148", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29848", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:21974", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:21950", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27830", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27838", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23227", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29422", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25179", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29272", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20221", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23239", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23257", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25885", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24886", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23826", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23201", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25822", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25009", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:icl-twobase10", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27902", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29837", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22127", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23288", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29315", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29492", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20114", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24867", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22176", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29779", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24876", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24872", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27855", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20151", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29313", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29325", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23868", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20335", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25138", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29893", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:21889", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22120", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29975", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22156", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29842", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22056", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24779", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24874", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25274", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29897", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23247", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24745", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20280", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23713", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22134", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24885", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:24893", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29291", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27701", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20293", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25919", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24830", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29809", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22021", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29883", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20244", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24889", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23895", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26038", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:24813", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23189", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27703", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29312", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23835", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20709", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26324", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25851", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23867", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29835", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20438", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25496", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25811", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20801", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25187", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29893", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20206", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23790", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25302", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20325", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20522", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22436", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28349", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26503", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23691", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22426", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20733", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25634", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24323", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20442", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20386", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22454", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26360", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:21906", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20680", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26474", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24234", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25632", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25308", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25372", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25438", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20305", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28222", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22499", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22487", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25277", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20304", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20500", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22599", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22562", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20813", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26468", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20642", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20747", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20440", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28315", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25359", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29982", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26418", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24283", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25312", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28318", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20356", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24296", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25523", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25325", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25757", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25354", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20810", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25310", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29774", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26488", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22432", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20679", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24237", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25303", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22420", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29939", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20625", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20364", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23651", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22610", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25277", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20644", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24343", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26322", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20385", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20218", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25620", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28238", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20407", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20783", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22607", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22549", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20630", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29833", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28363", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22543", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22616", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20655", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20124", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29821", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20046", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:tmophl7mts", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29844", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:24960", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25379", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24793", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29888", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29953", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:21897", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23725", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20134", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24685", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24705", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27750", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:24919", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29947", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:21987", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23202", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23841", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22064", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27885", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23357", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24871", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20078", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29967", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25923", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20355", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29825", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25383", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23296", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:24824", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22017", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25818", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29783", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20190", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23855", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24667", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27727", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27699", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24843", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:24840", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:21951", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29292", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24740", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24903", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:24930", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20136", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25177", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29314", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25827", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23170", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23295", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25254", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23743", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29921", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26057", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24824", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27819", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22148", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23347", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25157", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25013", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29757", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24766", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23807", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23924", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29758", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29260", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23818", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22042", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23331", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23341", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25925", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20132", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29963", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22082", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22113", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23400", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:novar-dbase", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25289", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:21991", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23687", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20061", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24852", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25345", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25927", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29872", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25934", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29776", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27742", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:21978", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20002", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:commtact-http", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29885", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:21952", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25653", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25677", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24354", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22534", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29775", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20302", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20532", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22645", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25648", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20499", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29846", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20643", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25369", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22422", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28161", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23659", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25472", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28236", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29756", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25337", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22638", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25397", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20472", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22555", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:vocaltec-wconf", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22561", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22380", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23863", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20418", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22413", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25403", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22390", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25641", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22410", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20333", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22392", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22398", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25455", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23841", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26426", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21248", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28857", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26183", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26044", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23148", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27250", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26007", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27129", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25487", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29825", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24285", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25661", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24288", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29818", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20355", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20374", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22600", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24257", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20623", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29989", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26338", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25027", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21589", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24862", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21550", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25384", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25457", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21478", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21640", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21684", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23561", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25071", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27582", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21880", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29332", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25007", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:icl-twobase8", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21957", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21629", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26422", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26651", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29545", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26496", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23633", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20785", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20438", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20724", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20790", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24261", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26217", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25983", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24301", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22977", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21267", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22852", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23093", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28810", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24234", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22914", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24824", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26994", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22900", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22968", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24216", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26105", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20870", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21270", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21591", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21561", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21606", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26679", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29425", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21595", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26717", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21711", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21497", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26543", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21497", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21387", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26569", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29478", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21636", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25824", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26766", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27141", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26947", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:24046", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26797", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27749", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21208", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27293", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21412", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23951", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21353", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22159", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25851", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27835", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25744", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25848", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23852", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29709", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25743", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29682", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22023", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25653", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27893", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26719", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21213", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26894", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:24076", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21749", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21744", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:24021", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27773", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27211", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21691", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27155", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26791", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27168", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27054", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27144", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25231", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21822", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:24017", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25825", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21898", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21318", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27899", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22255", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21724", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26801", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23860", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26699", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22029", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21795", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25706", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22195", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21765", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23838", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29726", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21740", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25792", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23929", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25726", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21942", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21874", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22158", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27194", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22220", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29744", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27087", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25670", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21988", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21651", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23977", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26769", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27250", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26771", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21801", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26773", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23927", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25719", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21322", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21650", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27138", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25679", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27974", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26702", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26920", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21188", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25785", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26813", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21380", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22078", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29607", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25318", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27946", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29814", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25631", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23886", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21797", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26771", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:24077", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27182", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22196", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29722", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29705", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25103", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25243", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21886", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27125", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23942", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:24083", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26749", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:24006", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:med-net-svc", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25671", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27786", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26829", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26898", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25834", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22918", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24182", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25735", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25870", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24300", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26928", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21062", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21292", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24641", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20255", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22991", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20324", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20731", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20395", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24695", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20935", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20680", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23066", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28804", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20862", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22845", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26906", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26107", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25875", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23023", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24254", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20909", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28723", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21030", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28648", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22895", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24767", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25915", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20709", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20824", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29697", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21688", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21180", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21899", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25837", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29632", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29777", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27157", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27197", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27973", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29752", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23903", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27913", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23908", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23936", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22239", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26915", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29765", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26711", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25139", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22049", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26671", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26734", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23967", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27914", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27131", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21832", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21852", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29604", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21712", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29760", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21701", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23980", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22144", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21264", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21835", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23900", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25689", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:24001", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:med-fsp-rx", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27176", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25756", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21830", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23960", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25096", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21675", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21802", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21199", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27791", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22126", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21300", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24306", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24311", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26794", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20712", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26882", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26326", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20782", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20864", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24283", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23000", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:inovaport1", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22850", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28769", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24325", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24143", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25717", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25205", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26831", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:24029", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27820", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21771", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21195", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29595", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29723", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23927", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23906", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27794", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22182", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26731", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23004", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:inovaport5", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29753", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21710", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25119", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27240", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21843", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29690", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21256", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25087", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25035", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29017", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26400", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26485", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24490", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21029", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23162", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23143", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21383", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23259", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24580", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26839", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21626", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21889", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26607", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23923", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:24009", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21183", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26590", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23801", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25097", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21777", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:22450", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27374", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27528", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:22033", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21519", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:28113", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:28200", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:voxelstorm", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26849", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29968", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:24248", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29859", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27310", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:24088", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25425", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:25460", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21950", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:24097", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29851", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:24278", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25463", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26583", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26675", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21145", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:27010", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25454", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23747", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27625", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21464", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29472", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23621", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20970", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27603", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25386", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25397", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25892", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29799", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29916", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27722", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29427", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29892", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:29946", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25262", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29840", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29811", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22098", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:25893", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20214", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27774", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29370", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24732", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25273", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:24900", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25142", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27713", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23747", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20015", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25307", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23771", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20135", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23698", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22168", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25304", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27898", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25003", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:icl-twobase4", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25200", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25242", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27845", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23307", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28128", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29839", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26502", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25796", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25292", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20219", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20483", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23681", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20375", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28149", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22464", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26532", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20491", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28365", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20318", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22592", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28336", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29919", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20304", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23834", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20468", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26336", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20808", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24319", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25719", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28184", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29881", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20458", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29836", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20467", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24328", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20755", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20787", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25793", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:vocaltec-hos", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25832", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25530", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25290", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25519", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25186", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25730", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25782", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20301", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25442", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22525", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25388", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25462", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20518", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20774", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26519", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26451", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29358", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25536", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26463", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26535", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21620", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21083", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26346", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29449", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24957", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28863", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:23792", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28685", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:23123", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28708", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28854", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:27625", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:25938", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28733", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20090", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22815", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23983", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24063", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22757", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20988", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26058", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20448", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22835", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20190", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26754", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20548", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25592", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25738", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23942", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23995", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:24946", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25449", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25554", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25584", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20038", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20177", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24131", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24150", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25108", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25178", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25256", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22203", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22344", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22358", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28008", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28108", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28122", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23477", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23488", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23536", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23540", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22297", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22328", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22364", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22379", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22390", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22397", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29576", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29591", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20405", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20419", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20459", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29377", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27660", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21025", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23789", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21135", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23733", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26402", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26992", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21656", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28662", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27100", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:29049", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:24126", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28866", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27206", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:28606", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:25889", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:24055", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:27706", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:29006", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:21577", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:26002", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:23086", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20864", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25441", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26700", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20897", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20951", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25596", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20120", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24437", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24445", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22822", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26641", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20019", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20986", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25474", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22783", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25937", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28549", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20156", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20210", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24058", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22897", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25579", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22735", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28431", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22837", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25010", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25122", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25409", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25441", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25606", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20066", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20116", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24024", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24051", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24095", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24156", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25088", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25112", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25134", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25169", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25219", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25257", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20152", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20177", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20180", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20262", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22190", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22231", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22284", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22338", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27912", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27951", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28020", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28093", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23426", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23578", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22208", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22210", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22409", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29613", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29654", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26066", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26104", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26134", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26246", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20489", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20513", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20516", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:21405", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27536", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26631", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27490", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26391", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26819", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:26700", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23688", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26549", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21037", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26573", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:21597", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:23634", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:21126", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:27481", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26470", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:26442", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25422", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:25483", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:29342", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:23648", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21878", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24878", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22788", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25469", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20559", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20123", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26637", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28525", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26638", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25614", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20459", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24629", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26024", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25981", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20639", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20756", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22776", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25504", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20657", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25618", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24584", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26636", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25401", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22774", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25583", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22669", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22742", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28465", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20673", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28541", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22839", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20001", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:microsan", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24033", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20177", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20583", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22726", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26730", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25425", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24599", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22842", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25559", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22778", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28578", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20000", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:dnp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26640", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28442", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20111", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26695", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20960", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25537", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25681", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24589", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21042", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26043", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20602", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20110", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25599", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20188", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24451", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28602", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20494", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24443", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24428", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20997", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20677", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22786", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26642", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20598", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20762", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24620", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20493", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28527", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20677", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20654", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22701", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20753", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25427", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24518", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22811", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20517", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25551", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26565", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22765", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20630", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20750", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20848", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20847", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20063", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26708", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24089", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29998", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:26038", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:24018", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22685", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22783", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21043", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20791", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28529", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21033", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25733", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28615", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20515", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20686", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24480", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20672", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20686", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24619", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24639", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20757", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22801", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25461", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22791", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21035", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28470", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:21030", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22685", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25866", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20105", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20220", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20938", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26739", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25875", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25918", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24474", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25872", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25768", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22825", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20073", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20771", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:20089", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25402", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25433", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25453", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25475", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25501", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25539", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20142", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20143", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20158", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24162", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25080", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25160", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25207", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20124", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20153", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20205", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22189", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22285", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22312", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22317", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22331", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22341", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27994", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28033", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23484", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23496", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23528", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23592", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22243", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22318", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22330", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22340", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26128", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26132", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26151", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26218", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20567", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25012", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25552", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25523", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20166", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24139", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22334", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:26074", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25119", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25515", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25101", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20209", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23445", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23470", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22274", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29600", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23941", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20404", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25003", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:icl-twobase4", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25082", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:25094", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25425", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25443", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25485", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25488", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25509", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:25534", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20072", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:20188", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:29991", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:23987", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24001", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:med-fsp-rx", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24025", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24065", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:24130", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25136", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25194", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:25270", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20178", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20213", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.75:20269", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22161", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22164", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22289", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.1.4:22327", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:27956", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:28025", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23571", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:23595", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22202", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:22411", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:29518", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20382", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20408", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:20556", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:28568", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:23100", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27089", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:29784", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27176", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:27535", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:21651", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:29781", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:21590", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:vofr-gateway", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27128", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27075", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:23099", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:27723", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27085", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27099", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:23197", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:28955", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:27760", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27077", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:27681", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:28591", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:29773", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28778", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:21622", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:28956", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:29175", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28798", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:29816", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28802", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27131", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.5:27084", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28781", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.68:29107", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:28601", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:28654", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:23571", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.5:23115", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:25861", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.69:28687", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28860", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:24089", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.16.2.5:23701", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.68:27740", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.69:28775", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "10.0.1.4:21490", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.5:25956", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "192.166.0.4:24059", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.80.36.0", "target": "172.18.0.4:29722", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.90.185.46", "target": "172.16.2.5:3443", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:ov-nnm-websrv", "connectionType": ""}, {"source": "20.43.139.112", "target": "172.18.0.4:10001", "src_to_dest_data_size": "4.62 MB", "dest_to_srct_data_size": "3.38 MB", "flowType": "AzurePublic", "protocol": "tcp:scp-config", "connectionType": ""}, {"source": "13.76.152.10", "target": "172.18.0.4:10001", "src_to_dest_data_size": "4.62 MB", "dest_to_srct_data_size": "3.38 MB", "flowType": "AzurePublic", "protocol": "tcp:scp-config", "connectionType": ""}, {"source": "104.215.128.10", "target": "192.166.0.4:10001", "src_to_dest_data_size": "3.63 MB", "dest_to_srct_data_size": "2.71 MB", "flowType": "AzurePublic", "protocol": "tcp:scp-config", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22647", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21584", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29509", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29634", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21750", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:27829", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21654", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25107", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23129", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20309", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23003", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:inovaport4", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20211", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20235", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21681", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24677", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:flashfiler", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23128", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25538", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23148", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:27921", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25522", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25349", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25335", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25354", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25187", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:27953", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23231", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25439", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21700", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:27965", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25355", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25213", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23150", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23152", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25325", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25358", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21736", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24702", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25445", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21749", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22689", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25287", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29641", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29786", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23485", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23007", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25843", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29947", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28498", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25795", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23482", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22183", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25875", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22083", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28268", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23111", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23524", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28398", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20620", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20819", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22127", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28377", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20764", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22273", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:wnn6", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23574", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23650", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23477", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29729", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28389", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25595", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23582", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25954", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25622", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22125", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:dcap", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23159", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29901", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28507", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23639", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25835", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22259", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28279", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25824", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29770", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23612", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22096", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28319", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22110", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29827", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29739", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25009", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:icl-twobase10", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22290", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29870", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28278", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23696", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21626", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20363", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25505", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22744", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:27913", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29346", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25266", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21762", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29378", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23017", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29426", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22729", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29437", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:27831", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22684", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23155", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21711", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29560", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23186", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21622", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24501", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22567", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20332", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29628", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24628", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29455", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29348", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:22922", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25108", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20295", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25227", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24505", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20192", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25459", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29465", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29298", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29466", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21657", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:27911", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29431", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22753", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29303", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22734", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23050", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23230", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25394", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29850", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28340", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29988", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25008", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:icl-twobase9", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23015", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23121", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20678", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22208", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25943", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28397", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23643", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23656", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26025", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23714", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29968", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23607", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22196", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25790", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23054", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28383", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29915", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25890", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22265", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25961", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22079", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29804", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28401", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21407", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26328", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20661", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23771", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22905", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29147", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:24249", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:vista-4gl", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21505", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20658", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25632", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23478", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20826", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25787", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20651", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20014", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:opendeploy", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20011", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20634", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26993", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24796", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:23410", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21901", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24715", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26896", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23367", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28898", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22577", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23672", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26296", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20370", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:24042", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21159", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29529", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21090", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:27143", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:20993", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21056", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:23352", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23229", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23395", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24793", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21848", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:netspeak-acd", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21004", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26246", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21192", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24569", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26466", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29732", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:27081", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:23510", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29791", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:27253", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24869", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:25042", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26414", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26535", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23574", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24921", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26625", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:27371", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23647", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:22191", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21322", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23589", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:25042", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23502", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24830", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23504", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:27179", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21336", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23528", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:25091", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:25043", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21333", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23604", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:27117", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:23573", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24831", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:23706", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24512", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:22200", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23433", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26472", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21271", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:25063", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26600", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:27216", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:25077", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:25028", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21227", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21237", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24832", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:27050", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26513", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:27041", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21363", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21515", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:22142", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21472", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:27288", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:25017", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21410", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:23639", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24591", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29922", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26389", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24653", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23475", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21411", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29831", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24853", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:25053", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21555", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:22171", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21412", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:27236", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26432", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24679", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:24021", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22623", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22556", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20541", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22506", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22696", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23969", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28829", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26429", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20568", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:20263", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20572", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23656", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26326", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20228", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23965", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22711", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25668", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20350", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:20414", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26150", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25532", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25664", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20407", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23947", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22771", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28876", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28895", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26116", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25456", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25517", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23492", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:27309", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:27323", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21280", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21334", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:27194", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:27240", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21200", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:27134", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:23717", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24610", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:23565", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23564", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24628", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21155", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21337", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:22229", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21244", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21387", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:24918", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29758", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21211", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26595", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24993", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28884", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20590", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20438", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22677", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:20261", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:20327", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20390", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21314", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:24145", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20516", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29956", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21256", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26626", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21492", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:23616", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22933", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23431", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:22049", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:27310", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23498", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23676", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23635", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:22101", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25778", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:20597", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20773", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20540", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26390", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23725", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:20430", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20698", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20791", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25823", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20774", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24526", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24659", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:24994", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:23622", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21273", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24496", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24592", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23596", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24974", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:25050", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:25354", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21678", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21665", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:25149", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21718", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21126", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29494", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26224", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21176", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26186", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21074", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24636", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21059", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22544", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:22976", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23094", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29292", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29328", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29319", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29399", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21520", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:27929", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29567", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29261", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29962", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22154", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20682", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22149", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23051", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23467", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22229", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25165", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22170", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23619", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29768", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20040", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29817", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29793", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22094", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22060", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28514", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22124", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21942", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29934", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28321", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25814", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23145", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29929", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29933", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22244", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22061", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:23303", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21068", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24669", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24670", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24410", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21296", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29480", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21194", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24806", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26369", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21922", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:27043", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:23118", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:26525", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:23340", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:23137", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:25511", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:24002", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:med-fsp-tx", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:26387", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25335", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28716", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20197", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23427", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23728", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22318", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22393", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20879", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22808", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21920", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25770", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21829", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21874", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24775", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24808", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24900", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20452", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25563", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29806", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:24716", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21110", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:24211", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26906", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:21038", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21254", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:24737", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29531", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:29668", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26978", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:24758", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:21076", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:26472", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:24132", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:26778", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:26944", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:26927", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:23238", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25963", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20058", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20219", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20154", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20355", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26204", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25229", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21081", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21069", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22920", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23314", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23406", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29523", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21917", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22017", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28142", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25601", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25723", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21782", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24751", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24804", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29671", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20434", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23296", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23332", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23356", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23368", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:24796", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:26287", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:21138", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26934", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26212", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26032", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23792", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26154", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25298", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20931", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20987", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23796", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23337", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20155", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25937", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26028", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25863", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20203", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22530", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26064", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25326", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:20131", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20190", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25888", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23282", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23841", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22314", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22382", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21064", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20139", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23937", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:20151", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28686", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26239", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25339", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26153", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20231", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:20339", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28560", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25887", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23865", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26186", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:20111", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23771", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20028", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25989", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26066", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25900", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:tasp-net", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20021", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26150", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23882", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23817", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:25236", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21001", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26175", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20886", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22262", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26010", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23695", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:26167", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20872", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:20147", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:22457", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28569", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22310", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:22264", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23795", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23687", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28697", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23733", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:26057", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28597", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:21046", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23475", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29948", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:23416", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29992", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23272", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23316", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23436", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23447", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23458", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21968", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25731", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21877", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21968", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.1.4:21971", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24802", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24812", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24815", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24884", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24939", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29637", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20398", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20513", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25419", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25477", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23371", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22814", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29525", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29644", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29567", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23362", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29611", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25636", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24859", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24866", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20550", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29564", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:29750", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29593", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28098", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22881", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22902", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:22957", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:23395", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:29627", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:21836", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28119", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28164", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:28191", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25632", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.75:25715", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:24829", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29591", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:29651", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20492", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:20504", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:25499", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23165", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23173", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23299", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:23376", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:23327", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:25478", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:23367", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:23151", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:26467", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:25555", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:28353", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:23245", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:23954", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:23326", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.5:23947", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:23333", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:elxmgmt", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:26508", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:28389", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:23042", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:29058", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:26787", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:23463", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.68:26944", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:26820", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:23164", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:23324", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:26458", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:23292", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:28986", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.4:23351", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:23108", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.16.2.5:23102", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.4:28523", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:28987", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:26556", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.69:21792", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.68:25599", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.5:29070", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "192.166.0.4:26839", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "172.18.0.69:26449", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:23084", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.192.0", "target": "10.0.1.5:23262", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "20.43.128.73", "target": "192.166.0.4:8443", "src_to_dest_data_size": "840 Bytes", "dest_to_srct_data_size": "264 Bytes", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "20.43.128.73", "target": "192.166.0.5:8444", "src_to_dest_data_size": "26.96 KB", "dest_to_srct_data_size": "26.88 KB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-http", "connectionType": ""}, {"source": "20.3.150.35", "target": "192.166.0.4:20000", "src_to_dest_data_size": "4.78 MB", "dest_to_srct_data_size": "23.1 MB", "flowType": "AzurePublic", "protocol": "tcp:dnp", "connectionType": ""}, {"source": "20.195.86.98", "target": "172.18.0.4:8443", "src_to_dest_data_size": "27.34 KB", "dest_to_srct_data_size": "26.61 KB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23170", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23296", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29248", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29867", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24200", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29015", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23283", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23278", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21815", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29113", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28185", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28226", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29008", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29102", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27014", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28135", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28164", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28199", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23149", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23229", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:26972", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23402", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:novar-global", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27189", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27493", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23378", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29158", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21704", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27147", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23113", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23294", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28102", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:29972", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28238", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23293", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:28989", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27301", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:29976", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:28981", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24176", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29909", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20120", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23262", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29003", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21614", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27116", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20212", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29894", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27045", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23251", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23226", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29095", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21745", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:29981", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27008", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23307", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29068", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29720", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20183", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29117", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:26990", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:29986", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20115", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24228", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24270", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29151", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20219", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23304", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29130", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29265", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23334", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29115", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20316", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21844", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27151", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27518", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28084", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29127", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27029", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23185", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27166", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20103", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20019", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20182", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29710", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29731", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24661", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23739", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20209", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29605", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29708", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20286", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27654", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28655", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28698", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29473", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27388", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20386", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20788", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28628", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22246", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24863", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20322", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24637", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20683", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27592", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29650", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22105", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27974", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20564", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23865", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20722", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28610", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20609", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24669", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23699", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29522", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29508", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27971", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29481", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29707", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22296", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29562", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23782", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29543", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29635", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23680", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29732", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29625", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23543", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20298", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29517", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27534", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28512", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23787", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27854", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23555", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29557", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20553", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28667", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23672", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23715", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20330", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20151", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24782", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23597", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20160", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20444", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23741", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29505", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29718", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23790", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20453", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24788", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20146", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22300", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24859", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20363", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27919", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29675", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22259", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20701", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27908", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23557", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28508", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23818", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29067", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23305", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29189", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27521", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27089", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21606", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23152", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23290", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20015", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29015", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29762", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21822", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27174", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20203", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23233", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23314", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29122", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29098", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27032", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27222", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24211", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21837", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20118", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20137", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24275", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29828", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29175", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27064", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24342", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21804", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20202", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:ipdtp-port", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21613", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23324", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29267", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29652", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21712", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24199", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23202", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23105", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28178", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27148", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21744", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24248", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23309", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24313", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28216", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27218", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29070", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29895", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29094", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29179", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20038", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23282", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23132", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20251", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20109", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29765", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27005", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24317", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27117", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29161", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:29993", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29722", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20232", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28228", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21671", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24271", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21708", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27528", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20337", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20197", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:29990", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20318", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23064", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29071", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:26962", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23234", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23308", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29157", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27342", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23111", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23208", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21740", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20198", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29229", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20058", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21622", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29244", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28148", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27048", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29079", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27204", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20425", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27423", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:26957", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20208", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24190", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21750", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23182", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27161", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20163", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29733", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29833", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23067", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:26924", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28132", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29212", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23318", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23241", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27063", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29091", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24167", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28141", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23310", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23260", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27248", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20056", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27860", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22187", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20503", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23674", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27590", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24878", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28000", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:nxlmd", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23801", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20263", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24840", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29724", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23591", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20615", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20134", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28483", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23721", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23685", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23630", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20321", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20518", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22140", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27677", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20458", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28498", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28624", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24708", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28605", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27973", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20550", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20582", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28529", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29525", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29642", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24875", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28524", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23752", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24745", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27701", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22139", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23762", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23791", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20717", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20789", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20173", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20274", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22827", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28308", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21480", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24339", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28167", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21226", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25371", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25584", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28515", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24401", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22954", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20245", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25447", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28254", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29328", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20175", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20179", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20505", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28615", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24738", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24824", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23764", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29572", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28546", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23678", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20262", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23569", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27620", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27935", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29526", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20251", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20877", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24649", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20304", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20516", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28482", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29613", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27818", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27587", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27899", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24759", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21393", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23498", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24897", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20779", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28711", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21887", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20757", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20865", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24886", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28738", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24863", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22074", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29197", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29774", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28568", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28808", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23367", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23352", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21966", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28656", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24853", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29150", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21365", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21035", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25307", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20942", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29084", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24330", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20102", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24207", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22762", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24347", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20773", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20039", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28000", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:nxlmd", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22587", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22793", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21152", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29039", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22732", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25298", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29078", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24776", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28745", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21997", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22033", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20932", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29795", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28990", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28596", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21422", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29866", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28839", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21903", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25818", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24891", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23329", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21558", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25880", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:21066", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26277", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25280", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28912", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22334", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21884", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25019", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22150", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22025", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22081", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21148", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29429", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25198", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29436", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25272", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25055", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25193", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22097", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20111", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25279", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20161", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26245", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20984", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25105", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:21000", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:irtrans", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20029", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28990", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23680", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21754", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23706", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20978", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21134", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21920", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25216", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22118", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23684", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20935", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25184", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28815", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22119", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26222", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20084", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25065", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25038", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21114", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21930", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:21013", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21687", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26283", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28973", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21057", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28828", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20087", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25070", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28989", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23685", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29024", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20922", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29084", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29224", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25114", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29126", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26190", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21951", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20943", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21172", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25220", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22004", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:optohost004", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26138", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25160", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21006", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28949", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:29040", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25207", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20988", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25236", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21697", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22111", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29095", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29145", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25081", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25130", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24989", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25126", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29036", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29078", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25150", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22073", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20890", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23586", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22075", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22111", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25060", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29048", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21934", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25207", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:21038", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29414", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20076", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20405", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20144", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21942", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22276", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28881", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25201", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21590", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:vofr-gateway", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21625", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21686", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20113", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21894", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22020", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21567", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21052", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29067", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20051", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29141", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22161", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21028", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28964", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22071", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29088", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21018", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28846", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21050", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29300", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24991", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21130", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22176", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25095", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29127", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21155", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29977", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25269", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25087", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29064", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29302", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23703", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24112", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29947", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24263", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24100", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27895", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27912", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21156", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24260", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28196", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20655", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20084", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29986", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20642", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28299", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28350", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21146", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22669", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28089", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27870", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21067", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28258", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25326", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29069", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24267", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24200", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20067", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20928", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20831", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20784", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29997", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24024", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29063", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24082", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24103", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21097", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25356", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29991", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22629", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27891", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28958", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20005", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:openwebnet", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29943", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29969", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22624", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28393", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25185", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22696", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24089", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21163", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21297", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20028", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25355", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24169", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20048", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:mountd", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27934", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21130", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28127", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23662", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21999", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26284", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26189", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26173", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20983", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20035", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26181", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:21041", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29309", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25296", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20991", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:21034", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22241", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20011", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20108", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21054", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25214", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:21093", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23614", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29990", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21604", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29425", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:29023", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20079", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:21125", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25122", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29986", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23651", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29017", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21585", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:29028", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25107", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26149", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25289", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25273", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29040", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25115", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25124", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25131", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23555", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29147", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21126", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29258", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28848", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21074", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29295", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:21032", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29236", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27999", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:tw-auth-key", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28062", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21360", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24189", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20999", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:athand-mmp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29958", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28070", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29204", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20041", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20964", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20967", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24152", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28080", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28167", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25199", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25284", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29153", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28939", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23719", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29277", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22246", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29991", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25006", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:icl-twobase7", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20915", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29008", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:29043", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28833", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22291", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24328", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22883", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28524", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20309", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25511", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21170", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28238", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28163", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21236", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29228", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21330", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24382", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22310", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21879", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25201", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25205", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20987", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29310", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21620", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25146", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25311", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28978", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25211", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23513", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23627", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21847", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:netspeak-cs", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:26422", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25406", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:21405", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:22165", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22544", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29616", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:29243", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:29220", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:29250", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23774", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29166", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:29202", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22552", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:25297", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:20222", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:ipulse-ics", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28725", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20775", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23510", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29891", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28861", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29837", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28621", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28748", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24812", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28760", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28851", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:21367", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:29808", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27015", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29757", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20225", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28223", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:28984", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27202", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29202", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23187", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20286", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27296", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27452", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28167", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27027", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29775", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20018", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20071", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23203", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23214", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20293", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24327", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23170", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24261", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29732", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29887", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27533", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27211", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29771", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20006", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21665", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27292", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20055", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20091", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23282", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21752", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24221", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29868", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20370", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29790", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21807", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24372", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:26912", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20013", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:ss-idi", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23300", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21823", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27501", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27542", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23810", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20708", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29543", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20744", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20650", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20267", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27539", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23704", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23593", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22173", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20902", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20800", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20834", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28728", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22270", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29513", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29699", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20281", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27575", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27380", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27527", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27914", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23819", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27488", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22310", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23713", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23767", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29487", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20724", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29713", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24798", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23609", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23798", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28684", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23736", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27533", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28606", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22096", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20246", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23681", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27540", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27670", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29677", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20566", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27547", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29693", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23682", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27566", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28678", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27641", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23615", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28008", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20569", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29653", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28656", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20687", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20737", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25969", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21875", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24896", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29101", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29106", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20831", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20691", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28969", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28689", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23415", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20781", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20828", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24930", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:20840", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:23717", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:20787", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:27038", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25662", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25530", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21342", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:23793", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:20882", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27109", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25534", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21335", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:22994", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:20862", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:27137", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25637", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28080", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:20748", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:22074", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:23902", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28165", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:21821", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:23613", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27912", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28912", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20533", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22439", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23975", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27724", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22435", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25088", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24091", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27862", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28930", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27782", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:ars-vista", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28741", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28781", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22448", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28931", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27853", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21051", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23493", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29322", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29418", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29452", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29973", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21881", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21963", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22024", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22027", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27284", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27318", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20463", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20550", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20581", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29329", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29404", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27622", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27316", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20234", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20273", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20288", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20437", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28358", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23508", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23527", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23561", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20719", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21946", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:22042", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21767", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24830", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:29020", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28622", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28642", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28567", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20663", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21651", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24797", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:20659", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28967", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24954", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25661", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:23609", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:21836", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27130", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:23047", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:23854", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28155", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:20744", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28071", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:23051", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:22932", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27100", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27205", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:26989", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:22857", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:23543", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21359", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28094", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28108", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27104", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20824", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28746", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24947", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29768", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29881", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27863", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27805", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24900", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22360", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20987", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27929", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21021", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28055", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24086", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22370", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29829", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24014", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23999", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28199", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27797", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23320", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23322", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23512", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23519", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29354", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29437", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29453", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29933", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29938", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21969", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27266", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27291", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27298", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27326", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29271", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29287", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29304", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29378", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29472", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29482", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27605", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27630", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27232", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20324", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20429", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20438", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24499", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28300", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28369", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28371", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28451", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23431", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23442", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23447", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23510", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28707", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21859", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25820", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25821", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:25039", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:24933", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:28685", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21709", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:23488", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21690", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25980", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21858", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25855", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21920", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:28754", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:21760", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29805", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23963", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23955", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20724", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24948", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27943", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28824", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24092", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22381", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20506", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23849", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20559", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29781", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24944", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24918", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23899", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21007", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27637", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29731", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20752", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20909", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27812", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28156", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29710", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24024", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20490", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25075", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23957", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22505", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22551", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20996", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23884", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28162", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23983", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23907", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20488", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29895", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23908", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29753", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23898", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21128", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20803", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29807", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27812", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24974", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20893", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20704", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27715", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23993", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22404", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27908", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29885", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20907", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23984", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27935", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20853", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20913", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29901", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28042", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24920", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20905", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28822", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28831", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24045", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20589", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27806", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20882", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20914", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23889", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24070", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29951", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29948", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27794", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23826", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21138", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27821", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24965", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28044", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20843", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29789", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29849", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29834", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29946", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27893", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29788", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22418", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22541", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29852", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23914", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29787", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20446", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20838", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25048", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27951", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20707", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28732", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21030", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29860", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27681", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27783", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27837", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21117", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24934", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20505", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:21018", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28073", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29873", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20874", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24084", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20554", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28864", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28125", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20733", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23906", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28813", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28934", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27922", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23813", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29865", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28195", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20841", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28872", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29941", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28150", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23887", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29982", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23943", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20831", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29714", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28222", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27685", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28877", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22420", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22457", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27808", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29768", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28897", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23994", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27767", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25109", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28057", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24023", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:25053", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27851", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:24078", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23865", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20790", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20496", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:22442", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27916", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:28161", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20798", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20472", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20535", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20855", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23437", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23490", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29328", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20016", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21880", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21902", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21909", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27402", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27411", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20442", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20464", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20574", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20589", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20622", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29286", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29290", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27552", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27606", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27620", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27755", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27181", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27328", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24410", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24542", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28256", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28271", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28464", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23443", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23495", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20245", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24526", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27654", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24446", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28299", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23476", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29291", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28455", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29339", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27228", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28359", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23395", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:29922", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21911", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20588", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27738", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20297", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20440", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23565", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20071", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21877", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27567", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:27716", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24431", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24593", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23343", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23390", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:23516", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:29343", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20002", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:commtact-http", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:20095", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21891", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21895", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21903", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:21935", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:27414", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20455", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.75:20628", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29279", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.1.4:29427", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27178", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27200", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27251", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:27295", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20327", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:20385", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:24427", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28297", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28351", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28382", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:28425", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23477", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23536", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:23547", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25645", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:27106", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:20840", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27105", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27155", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21168", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21151", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.5:24024", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28038", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27220", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:27091", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25425", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:20756", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28166", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25474", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21121", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:23649", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:21987", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:20884", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21316", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28051", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:23634", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25486", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:23582", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:23685", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:21833", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28130", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28164", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:23629", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:26941", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:26985", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.4:21910", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:20938", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:20928", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:27121", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.16.2.5:23593", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:27988", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27158", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21331", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25574", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:20964", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27179", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:26929", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:27134", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25587", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:27111", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:20762", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.5:20918", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28125", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21212", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27093", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.4:27143", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:23033", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21268", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.68:25653", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.69:27100", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.5:28099", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:22948", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "192.166.0.69:21015", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21285", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "172.18.0.68:22991", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "40.87.200.0", "target": "10.0.1.4:21303", "src_to_dest_data_size": "128 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp", "connectionType": ""}, {"source": "20.195.86.100", "target": "172.18.0.5:8444", "src_to_dest_data_size": "28.48 KB", "dest_to_srct_data_size": "27.29 KB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-http", "connectionType": ""}, {"source": "20.195.86.100", "target": "192.166.0.4:8443", "src_to_dest_data_size": "28.93 KB", "dest_to_srct_data_size": "29.88 KB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "20.195.86.100", "target": "172.18.0.4:8443", "src_to_dest_data_size": "27.36 KB", "dest_to_srct_data_size": "27.96 KB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "20.195.86.100", "target": "192.166.0.5:8444", "src_to_dest_data_size": "28.46 KB", "dest_to_srct_data_size": "28.38 KB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-http", "connectionType": ""}, {"source": "52.163.205.184", "target": "52.187.163.69:500", "src_to_dest_data_size": "40.7 KB", "dest_to_srct_data_size": "30.08 KB", "flowType": "AzurePublic", "protocol": "udp:isakmp", "connectionType": ""}, {"source": "20.43.136.204", "target": "192.166.0.4:10001", "src_to_dest_data_size": "3.63 MB", "dest_to_srct_data_size": "2.71 MB", "flowType": "AzurePublic", "protocol": "tcp:scp-config", "connectionType": ""}, {"source": "20.84.119.140", "target": "172.18.0.4:8443", "src_to_dest_data_size": "515.2 KB", "dest_to_srct_data_size": "1.93 MB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "20.84.119.140", "target": "172.18.0.4:20000", "src_to_dest_data_size": "17.24 KB", "dest_to_srct_data_size": "23.59 KB", "flowType": "AzurePublic", "protocol": "tcp:dnp", "connectionType": ""}, {"source": "20.84.119.140", "target": "52.163.205.184:500", "src_to_dest_data_size": "510 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "udp:isakmp", "connectionType": ""}, {"source": "20.55.51.23", "target": "172.18.0.4:8082", "src_to_dest_data_size": "20.68 KB", "dest_to_srct_data_size": "25.86 KB", "flowType": "AzurePublic", "protocol": "tcp:us-cli", "connectionType": ""}, {"source": "20.55.51.23", "target": "172.18.0.4:10001", "src_to_dest_data_size": "17.17 KB", "dest_to_srct_data_size": "21.13 KB", "flowType": "AzurePublic", "protocol": "tcp:scp-config", "connectionType": ""}, {"source": "20.163.15.225", "target": "192.166.0.4:8443", "src_to_dest_data_size": "1.33 KB", "dest_to_srct_data_size": "5.2 KB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "172.179.233.147", "target": "192.166.0.68:443", "src_to_dest_data_size": "7.29 KB", "dest_to_srct_data_size": "25.72 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.195.86.96", "target": "192.166.0.4:20000", "src_to_dest_data_size": "7.02 KB", "dest_to_srct_data_size": "15.01 KB", "flowType": "AzurePublic", "protocol": "tcp:dnp", "connectionType": ""}, {"source": "20.84.52.103", "target": "10.0.1.4:443", "src_to_dest_data_size": "8.58 KB", "dest_to_srct_data_size": "26.9 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.84.52.103", "target": "10.0.1.5:443", "src_to_dest_data_size": "27.01 KB", "dest_to_srct_data_size": "69.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.65.193.168", "target": "192.166.0.68:443", "src_to_dest_data_size": "1.92 KB", "dest_to_srct_data_size": "6.82 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.65.193.168", "target": "172.18.0.69:443", "src_to_dest_data_size": "204 Bytes", "dest_to_srct_data_size": "60 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.65.193.168", "target": "172.18.0.68:443", "src_to_dest_data_size": "1.72 KB", "dest_to_srct_data_size": "6.64 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "13.67.64.10", "target": "172.18.0.4:10001", "src_to_dest_data_size": "4.61 MB", "dest_to_srct_data_size": "3.38 MB", "flowType": "AzurePublic", "protocol": "tcp:scp-config", "connectionType": ""}, {"source": "4.242.34.174", "target": "172.18.0.69:443", "src_to_dest_data_size": "30.78 KB", "dest_to_srct_data_size": "72.42 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "4.242.34.174", "target": "172.18.0.68:443", "src_to_dest_data_size": "4.88 KB", "dest_to_srct_data_size": "11.48 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "13.65.25.145", "target": "172.18.0.4:8081", "src_to_dest_data_size": "13.21 MB", "dest_to_srct_data_size": "49.21 MB", "flowType": "AzurePublic", "protocol": "tcp:sunproxyadmin", "connectionType": ""}, {"source": "13.65.25.145", "target": "172.18.0.4:10002", "src_to_dest_data_size": "11.94 MB", "dest_to_srct_data_size": "54.59 MB", "flowType": "AzurePublic", "protocol": "tcp:documentum", "connectionType": ""}, {"source": "13.65.25.145", "target": "172.18.0.4:8082", "src_to_dest_data_size": "8.46 MB", "dest_to_srct_data_size": "31.73 MB", "flowType": "AzurePublic", "protocol": "tcp:us-cli", "connectionType": ""}, {"source": "13.65.25.145", "target": "172.18.0.4:20000", "src_to_dest_data_size": "11.16 MB", "dest_to_srct_data_size": "50.6 MB", "flowType": "AzurePublic", "protocol": "tcp:dnp", "connectionType": ""}, {"source": "13.65.25.145", "target": "172.18.0.4:10001", "src_to_dest_data_size": "12.32 MB", "dest_to_srct_data_size": "51.25 MB", "flowType": "AzurePublic", "protocol": "tcp:scp-config", "connectionType": ""}, {"source": "13.65.25.145", "target": "172.18.0.5:8444", "src_to_dest_data_size": "1.07 MB", "dest_to_srct_data_size": "4.87 MB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-http", "connectionType": ""}, {"source": "13.65.25.145", "target": "172.18.0.4:8443", "src_to_dest_data_size": "215.28 KB", "dest_to_srct_data_size": "593.87 KB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "20.115.226.253", "target": "172.18.0.69:22", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:ssh", "connectionType": ""}, {"source": "20.98.153.19", "target": "192.166.0.4:8443", "src_to_dest_data_size": "562 Bytes", "dest_to_srct_data_size": "186 Bytes", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "20.98.153.19", "target": "172.18.0.4:8443", "src_to_dest_data_size": "563 Bytes", "dest_to_srct_data_size": "186 Bytes", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "20.84.119.144", "target": "172.18.0.68:443", "src_to_dest_data_size": "9.7 MB", "dest_to_srct_data_size": "21.23 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.84.119.144", "target": "172.18.0.69:443", "src_to_dest_data_size": "9.9 MB", "dest_to_srct_data_size": "22.97 MB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.84.119.144", "target": "172.18.0.69:22", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:ssh", "connectionType": ""}, {"source": "20.84.119.144", "target": "172.18.0.68:22", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:ssh", "connectionType": ""}, {"source": "20.169.49.63", "target": "10.0.1.5:443", "src_to_dest_data_size": "4.33 KB", "dest_to_srct_data_size": "22 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.169.49.63", "target": "10.0.1.4:443", "src_to_dest_data_size": "1.05 KB", "dest_to_srct_data_size": "279 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "52.180.144.125", "target": "192.166.0.68:443", "src_to_dest_data_size": "5.43 KB", "dest_to_srct_data_size": "22.27 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "52.180.144.125", "target": "192.166.0.69:443", "src_to_dest_data_size": "5.49 KB", "dest_to_srct_data_size": "1.6 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "52.180.144.125", "target": "172.18.0.69:443", "src_to_dest_data_size": "10.62 KB", "dest_to_srct_data_size": "23.81 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "52.180.144.125", "target": "172.18.0.68:443", "src_to_dest_data_size": "204 Bytes", "dest_to_srct_data_size": "60 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "13.65.254.147", "target": "192.166.0.4:10001", "src_to_dest_data_size": "865.56 KB", "dest_to_srct_data_size": "3.05 MB", "flowType": "AzurePublic", "protocol": "tcp:scp-config", "connectionType": ""}, {"source": "13.65.254.147", "target": "192.166.0.4:10002", "src_to_dest_data_size": "865.67 KB", "dest_to_srct_data_size": "3.42 MB", "flowType": "AzurePublic", "protocol": "tcp:documentum", "connectionType": ""}, {"source": "13.65.254.147", "target": "52.187.163.69:443", "src_to_dest_data_size": "3.09 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "13.65.254.147", "target": "192.166.0.4:179", "src_to_dest_data_size": "2.89 KB", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:bgp", "connectionType": ""}, {"source": "13.65.254.147", "target": "192.166.0.4:20000", "src_to_dest_data_size": "867.71 KB", "dest_to_srct_data_size": "3.42 MB", "flowType": "AzurePublic", "protocol": "tcp:dnp", "connectionType": ""}, {"source": "20.127.153.255", "target": "10.0.1.5:22", "src_to_dest_data_size": "0 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:ssh", "connectionType": ""}, {"source": "4.151.124.52", "target": "192.166.0.4:8443", "src_to_dest_data_size": "6.56 KB", "dest_to_srct_data_size": "11.59 KB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "4.151.124.52", "target": "192.166.0.5:8444", "src_to_dest_data_size": "32.03 KB", "dest_to_srct_data_size": "71.6 KB", "flowType": "AzurePublic", "protocol": "tcp:pcsync-http", "connectionType": ""}, {"source": "20.168.0.86", "target": "192.166.0.69:443", "src_to_dest_data_size": "574 Bytes", "dest_to_srct_data_size": "272 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.168.0.86", "target": "172.18.0.68:443", "src_to_dest_data_size": "466 Bytes", "dest_to_srct_data_size": "272 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.168.0.86", "target": "192.166.0.68:443", "src_to_dest_data_size": "204 Bytes", "dest_to_srct_data_size": "60 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.168.0.86", "target": "172.18.0.69:443", "src_to_dest_data_size": "204 Bytes", "dest_to_srct_data_size": "60 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.98.139.16", "target": "52.187.163.69:500", "src_to_dest_data_size": "444 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "udp:isakmp", "connectionType": ""}, {"source": "20.65.194.9", "target": "172.18.0.68:443", "src_to_dest_data_size": "1.44 KB", "dest_to_srct_data_size": "6.64 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.65.194.9", "target": "192.166.0.69:443", "src_to_dest_data_size": "1.44 KB", "dest_to_srct_data_size": "6.64 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.65.194.9", "target": "192.166.0.68:443", "src_to_dest_data_size": "204 Bytes", "dest_to_srct_data_size": "60 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.188.64.187", "target": "192.166.0.4:8082", "src_to_dest_data_size": "28.51 KB", "dest_to_srct_data_size": "62.32 KB", "flowType": "AzurePublic", "protocol": "tcp:us-cli", "connectionType": ""}, {"source": "20.188.64.187", "target": "192.166.0.4:8081", "src_to_dest_data_size": "2.25 KB", "dest_to_srct_data_size": "952 Bytes", "flowType": "AzurePublic", "protocol": "tcp:sunproxyadmin", "connectionType": ""}, {"source": "40.124.172.100", "target": "10.0.1.5:443", "src_to_dest_data_size": "572 Bytes", "dest_to_srct_data_size": "272 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.171.28.183", "target": "192.166.0.4:8443", "src_to_dest_data_size": "651 Bytes", "dest_to_srct_data_size": "186 Bytes", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "20.171.28.183", "target": "172.18.0.4:8443", "src_to_dest_data_size": "652 Bytes", "dest_to_srct_data_size": "186 Bytes", "flowType": "AzurePublic", "protocol": "tcp:pcsync-https", "connectionType": ""}, {"source": "4.227.179.212", "target": "192.166.0.4:10002", "src_to_dest_data_size": "19.35 KB", "dest_to_srct_data_size": "41.48 KB", "flowType": "AzurePublic", "protocol": "tcp:documentum", "connectionType": ""}, {"source": "4.227.179.212", "target": "192.166.0.4:10001", "src_to_dest_data_size": "102 Bytes", "dest_to_srct_data_size": "232 Bytes", "flowType": "AzurePublic", "protocol": "tcp:scp-config", "connectionType": ""}, {"source": "20.118.240.163", "target": "52.163.205.184:443", "src_to_dest_data_size": "102 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.118.240.163", "target": "52.187.163.69:443", "src_to_dest_data_size": "102 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.65.193.234", "target": "10.0.1.4:443", "src_to_dest_data_size": "204 Bytes", "dest_to_srct_data_size": "60 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.65.193.234", "target": "10.0.1.5:443", "src_to_dest_data_size": "1.34 KB", "dest_to_srct_data_size": "5.68 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "135.119.17.200", "target": "52.163.205.184:443", "src_to_dest_data_size": "102 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.14.80.89", "target": "10.0.1.4:443", "src_to_dest_data_size": "1.44 KB", "dest_to_srct_data_size": "6.64 KB", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "20.150.201.97", "target": "52.187.163.69:443", "src_to_dest_data_size": "102 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}, {"source": "4.150.187.93", "target": "192.166.0.4:8081", "src_to_dest_data_size": "562 Bytes", "dest_to_srct_data_size": "186 Bytes", "flowType": "AzurePublic", "protocol": "tcp:sunproxyadmin", "connectionType": ""}, {"source": "20.115.212.5", "target": "52.187.163.69:443", "src_to_dest_data_size": "102 Bytes", "dest_to_srct_data_size": "0 Bytes", "flowType": "AzurePublic", "protocol": "tcp:https", "connectionType": ""}], "categories": "[{\"name\":\"AzurePublic\"},{\"name\":\"ExternalPublic\"},{\"name\":\"UnknownPrivate\"},{\"name\":\"InterVNet\"},{\"name\":\"IntraVNet\"},{\"name\":\"MaliciousFlow\"}]"}


### source info for grafana filter